In [151]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import collections
from collections import OrderedDict
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

In [179]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

####### Macro Cases ######
dfn2_macro_fused_probempi = pd.read_csv("n2_fused_macro_probmpi.csv")
dfn2_macro_fused_nonprobempi = pd.read_csv("n2_fused_macro_nonprobmpi.csv")

dfn2_macro_newgeo_probempi = pd.read_csv("n2_newgeo_macro_probmpi.csv")
dfn2_macro_newgeo_nonprobempi = pd.read_csv("n2_newgeo_macro_nonprobmpi.csv")

dfn2_macro_nonfused_probempi = pd.read_csv("n2_nonfused_macro_probmpi.csv")
dfn2_macro_nonfused_nonprobempi = pd.read_csv("n2_nonfused_macro_nonprobmpi.csv")

###### AVX Cases ######
dfn2_avx_fused_probmpi = pd.read_csv("n2_fused_avx_probmpi.csv")
dfn2_avx_fused_nonprobmpi = pd.read_csv("n2_fused_avx_nonprobmpi.csv")

dfn2_avx_newgeo_probempi = pd.read_csv("n2_newgeo_avx_probmpi.csv")
dfn2_avx_newgeo_nonprobempi = pd.read_csv("n2_newgeo_avx_nonprobmpi.csv")

dfn2_avx_nonfused_probempi = pd.read_csv("n2_nonfused_avx_probmpi.csv")
dfn2_avx_nonfused_nonprobempi = pd.read_csv("n2_nonfused_avx_nonprobmpi.csv")

####### VECT Cases ######
dfn2_vect_fused_probmpi = pd.read_csv("n2_fused_vect_probmpi.csv")
dfn2_vect_fused_nonprobmpi = pd.read_csv("n2_fused_vect_nonprobmpi.csv")

dfn2_vect_newgeo_probempi = pd.read_csv("n2_newgeo_vect_probmpi.csv")
dfn2_vect_newgeo_nonprobempi = pd.read_csv("n2_newgeo_vect_nonprobmpi.csv")

dfn2_vect_nonfused_probempi = pd.read_csv("n2_nonfused_vect_probmpi.csv")
dfn2_vect_nonfused_nonprobempi = pd.read_csv("n2_nonfused_vect_nonprobmpi.csv")



# HIPPO MACRO CASES

## 1) Fused_Macro_Probempi

### Adding Columns

In [62]:
dfn2_macro_fused_probempi["N_Nodes"] = 2
dfn2_macro_fused_probempi['NPX'] = (dfn2_macro_fused_probempi.GLB_X//dfn2_macro_fused_probempi.LOC_X)
dfn2_macro_fused_probempi['NPY'] = (dfn2_macro_fused_probempi.GLB_Y//dfn2_macro_fused_probempi.LOC_Y)
dfn2_macro_fused_probempi['NPZ'] = (dfn2_macro_fused_probempi.GLB_Z//dfn2_macro_fused_probempi.LOC_Z)
dfn2_macro_fused_probempi['NPT'] = (dfn2_macro_fused_probempi.GLB_T//dfn2_macro_fused_probempi.LOC_T)
dfn2_macro_fused_probempi["Total_proc"] = dfn2_macro_fused_probempi["omp_thrd"]* dfn2_macro_fused_probempi.MPI_size
# dfn2_macro_fused_probempi.columns

### Data Summary 

In [63]:
dfn2_macro_fused_probempi_summary = dfn2_macro_fused_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn2_macro_fused_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,16,16,2,2,2,2,256,140
1,32,8,4,2,2,2,256,70
2,64,4,8,2,2,2,256,35
3,128,2,16,2,2,2,256,17
4,248,1,31,2,2,2,248,4
5,256,1,32,2,2,2,256,7


### Cases Setup

In [64]:
dfn2MacFusProb_16mpi_16omp = dfn2_macro_fused_probempi[(dfn2_macro_fused_probempi.omp_thrd == 16) & (dfn2_macro_fused_probempi.GLB_T/dfn2_macro_fused_probempi.LOC_T == 2) & (dfn2_macro_fused_probempi.GLB_X/dfn2_macro_fused_probempi.LOC_X == 2) & (dfn2_macro_fused_probempi.GLB_Y/dfn2_macro_fused_probempi.LOC_Y == 2)& (dfn2_macro_fused_probempi.GLB_Z/dfn2_macro_fused_probempi.LOC_Z == 2)]
dfn2MacFusProb_32mpi_8omp = dfn2_macro_fused_probempi[(dfn2_macro_fused_probempi.omp_thrd == 8) & (dfn2_macro_fused_probempi.GLB_T/dfn2_macro_fused_probempi.LOC_T == 4) & (dfn2_macro_fused_probempi.GLB_X/dfn2_macro_fused_probempi.LOC_X == 2) & (dfn2_macro_fused_probempi.GLB_Y/dfn2_macro_fused_probempi.LOC_Y == 2)& (dfn2_macro_fused_probempi.GLB_Z/dfn2_macro_fused_probempi.LOC_Z == 2)]
dfn2MacFusProb_64mpi_4omp = dfn2_macro_fused_probempi[(dfn2_macro_fused_probempi.omp_thrd == 4) & (dfn2_macro_fused_probempi.GLB_T/dfn2_macro_fused_probempi.LOC_T == 8) & (dfn2_macro_fused_probempi.GLB_X/dfn2_macro_fused_probempi.LOC_X == 2) & (dfn2_macro_fused_probempi.GLB_Y/dfn2_macro_fused_probempi.LOC_Y == 2)& (dfn2_macro_fused_probempi.GLB_Z/dfn2_macro_fused_probempi.LOC_Z == 2)]
dfn2MacFusProb_128mpi_2omp = dfn2_macro_fused_probempi[(dfn2_macro_fused_probempi.omp_thrd == 2) & (dfn2_macro_fused_probempi.GLB_T/dfn2_macro_fused_probempi.LOC_T == 16) & (dfn2_macro_fused_probempi.GLB_X/dfn2_macro_fused_probempi.LOC_X == 2) & (dfn2_macro_fused_probempi.GLB_Y/dfn2_macro_fused_probempi.LOC_Y == 2)& (dfn2_macro_fused_probempi.GLB_Z/dfn2_macro_fused_probempi.LOC_Z == 2)]
dfn2MacFusProb_256mpi_1omp = dfn2_macro_fused_probempi[(dfn2_macro_fused_probempi.omp_thrd == 1) & (dfn2_macro_fused_probempi.GLB_T/dfn2_macro_fused_probempi.LOC_T == 32) & (dfn2_macro_fused_probempi.GLB_X/dfn2_macro_fused_probempi.LOC_X == 2) & (dfn2_macro_fused_probempi.GLB_Y/dfn2_macro_fused_probempi.LOC_Y == 2)& (dfn2_macro_fused_probempi.GLB_Z/dfn2_macro_fused_probempi.LOC_Z == 2)]



## 2) Fused_Macro_nonprobempi

### Adding Columns

In [65]:
dfn2_macro_fused_nonprobempi["N_Nodes"] = 2
dfn2_macro_fused_nonprobempi['NPX'] = (dfn2_macro_fused_nonprobempi.GLB_X//dfn2_macro_fused_nonprobempi.LOC_X)
dfn2_macro_fused_nonprobempi['NPY'] = (dfn2_macro_fused_nonprobempi.GLB_Y//dfn2_macro_fused_nonprobempi.LOC_Y)
dfn2_macro_fused_nonprobempi['NPZ'] = (dfn2_macro_fused_nonprobempi.GLB_Z//dfn2_macro_fused_nonprobempi.LOC_Z)
dfn2_macro_fused_nonprobempi['NPT'] = (dfn2_macro_fused_nonprobempi.GLB_T//dfn2_macro_fused_nonprobempi.LOC_T)
dfn2_macro_fused_nonprobempi["Total_proc"] = dfn2_macro_fused_nonprobempi["omp_thrd"]* dfn2_macro_fused_nonprobempi.MPI_size
# dfn2_macro_fused_nonprobempi.columns

### Data Summary

In [66]:
dfn2_macro_fused_nonprobempi_summary =dfn2_macro_fused_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn2_macro_fused_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,16,16,2,2,2,2,256,138
1,32,8,4,2,2,2,256,70
2,64,4,8,2,2,2,256,35
3,128,2,16,2,2,2,256,17
4,248,1,31,2,2,2,248,4
5,256,1,32,2,2,2,256,7


### Cases Setup 

In [67]:
dfn2MacFusNonProb_16mpi_16omp = dfn2_macro_fused_nonprobempi[(dfn2_macro_fused_nonprobempi.omp_thrd == 16) & (dfn2_macro_fused_nonprobempi.GLB_T/dfn2_macro_fused_nonprobempi.LOC_T == 2) & (dfn2_macro_fused_nonprobempi.GLB_X/dfn2_macro_fused_nonprobempi.LOC_X == 2) & (dfn2_macro_fused_nonprobempi.GLB_Y/dfn2_macro_fused_nonprobempi.LOC_Y == 2)& (dfn2_macro_fused_nonprobempi.GLB_Z/dfn2_macro_fused_nonprobempi.LOC_Z == 2)]
dfn2MacFusNonProb_32mpi_8omp = dfn2_macro_fused_nonprobempi[(dfn2_macro_fused_nonprobempi.omp_thrd == 8) & (dfn2_macro_fused_nonprobempi.GLB_T/dfn2_macro_fused_nonprobempi.LOC_T == 4) & (dfn2_macro_fused_nonprobempi.GLB_X/dfn2_macro_fused_nonprobempi.LOC_X == 2) & (dfn2_macro_fused_nonprobempi.GLB_Y/dfn2_macro_fused_nonprobempi.LOC_Y == 2)& (dfn2_macro_fused_nonprobempi.GLB_Z/dfn2_macro_fused_nonprobempi.LOC_Z == 2)]
dfn2MacFusnonProb_64mpi_4omp = dfn2_macro_fused_nonprobempi[(dfn2_macro_fused_nonprobempi.omp_thrd == 4) & (dfn2_macro_fused_nonprobempi.GLB_T/dfn2_macro_fused_nonprobempi.LOC_T == 8) & (dfn2_macro_fused_nonprobempi.GLB_X/dfn2_macro_fused_nonprobempi.LOC_X == 2) & (dfn2_macro_fused_nonprobempi.GLB_Y/dfn2_macro_fused_nonprobempi.LOC_Y == 2)& (dfn2_macro_fused_nonprobempi.GLB_Z/dfn2_macro_fused_nonprobempi.LOC_Z == 2)]
dfn2MacFusnonProb_128mpi_2omp = dfn2_macro_fused_nonprobempi[(dfn2_macro_fused_nonprobempi.omp_thrd == 2) & (dfn2_macro_fused_nonprobempi.GLB_T/dfn2_macro_fused_nonprobempi.LOC_T == 16) & (dfn2_macro_fused_nonprobempi.GLB_X/dfn2_macro_fused_nonprobempi.LOC_X == 2) & (dfn2_macro_fused_nonprobempi.GLB_Y/dfn2_macro_fused_nonprobempi.LOC_Y == 2)& (dfn2_macro_fused_nonprobempi.GLB_Z/dfn2_macro_fused_nonprobempi.LOC_Z == 2)]
dfn2MacFusnonProb_256mpi_1omp = dfn2_macro_fused_nonprobempi[(dfn2_macro_fused_nonprobempi.omp_thrd == 1) & (dfn2_macro_fused_nonprobempi.GLB_T/dfn2_macro_fused_nonprobempi.LOC_T == 32) & (dfn2_macro_fused_nonprobempi.GLB_X/dfn2_macro_fused_nonprobempi.LOC_X == 2) & (dfn2_macro_fused_nonprobempi.GLB_Y/dfn2_macro_fused_nonprobempi.LOC_Y == 2)& (dfn2_macro_fused_nonprobempi.GLB_Z/dfn2_macro_fused_nonprobempi.LOC_Z == 2)]


## GFLOPS - Fused Macro: Probempi vs NonProbempi

In [68]:
fig = go.Figure() 

fig.add_trace(go.Scatter(x=(dfn2MacFusnonProb_256mpi_1omp.Dirc_Data_Mov*(dfn2MacFusnonProb_256mpi_1omp.MPI_size/2)), 
                         y=(dfn2MacFusnonProb_256mpi_1omp.GFLOPS*(dfn2MacFusnonProb_256mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacFusnonProb_256mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


fig.add_trace(go.Scatter(x=(dfn2MacFusProb_256mpi_1omp.Dirc_Data_Mov*(dfn2MacFusProb_256mpi_1omp.MPI_size/2)), 
                         y=(dfn2MacFusProb_256mpi_1omp.GFLOPS*(dfn2MacFusProb_256mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacFusProb_256mpi_1omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

# case 2
fig.add_trace(go.Scatter(x=(dfn2MacFusnonProb_128mpi_2omp.Dirc_Data_Mov*(dfn2MacFusnonProb_128mpi_2omp.MPI_size/2)), 
                         y=(dfn2MacFusnonProb_128mpi_2omp.GFLOPS*(dfn2MacFusnonProb_128mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacFusnonProb_128mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn2MacFusProb_128mpi_2omp.Dirc_Data_Mov*(dfn2MacFusProb_128mpi_2omp.MPI_size/2)), 
                         y=(dfn2MacFusProb_128mpi_2omp.GFLOPS*(dfn2MacFusProb_128mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacFusProb_128mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

# case 3
fig.add_trace(go.Scatter(x=(dfn2MacFusnonProb_64mpi_4omp.Dirc_Data_Mov*(dfn2MacFusnonProb_64mpi_4omp.MPI_size/2)), 
                         y=(dfn2MacFusnonProb_64mpi_4omp.GFLOPS*(dfn2MacFusnonProb_64mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacFusnonProb_64mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))

fig.add_trace(go.Scatter(x=(dfn2MacFusProb_64mpi_4omp.Dirc_Data_Mov*(dfn2MacFusProb_64mpi_4omp.MPI_size/2)), 
                         y=(dfn2MacFusProb_64mpi_4omp.GFLOPS*(dfn2MacFusProb_64mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacFusProb_64mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

# case 4
fig.add_trace(go.Scatter(x=(dfn2MacFusNonProb_32mpi_8omp.Dirc_Data_Mov*(dfn2MacFusNonProb_32mpi_8omp.MPI_size/2)), 
                         y=(dfn2MacFusNonProb_32mpi_8omp.GFLOPS*(dfn2MacFusNonProb_32mpi_8omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacFusNonProb_32mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='red',width=2))))

fig.add_trace(go.Scatter(x=(dfn2MacFusProb_32mpi_8omp.Dirc_Data_Mov*(dfn2MacFusProb_32mpi_8omp.MPI_size/2)), 
                         y=(dfn2MacFusProb_32mpi_8omp.GFLOPS*(dfn2MacFusProb_32mpi_8omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacFusProb_32mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='green',width=2))))

# case 5

fig.add_trace(go.Scatter(x=(dfn2MacFusNonProb_16mpi_16omp.Dirc_Data_Mov*(dfn2MacFusNonProb_16mpi_16omp.MPI_size/2)), 
                         y=(dfn2MacFusNonProb_16mpi_16omp.GFLOPS*(dfn2MacFusNonProb_16mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacFusNonProb_16mpi_16omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='chocolate',width=2))))

fig.add_trace(go.Scatter(x=(dfn2MacFusProb_16mpi_16omp.Dirc_Data_Mov*(dfn2MacFusProb_16mpi_16omp.MPI_size/2)), 
                         y=(dfn2MacFusProb_16mpi_16omp.GFLOPS*(dfn2MacFusProb_16mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacFusProb_16mpi_16omp',
                         line=dict(color='darkviolet',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='crimson',width=2))))



# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"Fused-Macro (2-Nodes): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1))

## 3) Nonfused_Macro_Probempi

### Adding Columns

In [69]:
dfn2_macro_nonfused_probempi["N_Nodes"] = 2
dfn2_macro_nonfused_probempi['NPX'] = (dfn2_macro_nonfused_probempi.GLB_X//dfn2_macro_nonfused_probempi.LOC_X)
dfn2_macro_nonfused_probempi['NPY'] = (dfn2_macro_nonfused_probempi.GLB_Y//dfn2_macro_nonfused_probempi.LOC_Y)
dfn2_macro_nonfused_probempi['NPZ'] = (dfn2_macro_nonfused_probempi.GLB_Z//dfn2_macro_nonfused_probempi.LOC_Z)
dfn2_macro_nonfused_probempi['NPT'] = (dfn2_macro_nonfused_probempi.GLB_T//dfn2_macro_nonfused_probempi.LOC_T)
dfn2_macro_nonfused_probempi["Total_proc"] = dfn2_macro_nonfused_probempi["omp_thrd"]* dfn2_macro_nonfused_probempi.MPI_size
# dfn2_macro_nonfused_probempi.columns

### Data Summary

In [70]:
dfn2_macro_nonfused_probempi_summary = dfn2_macro_nonfused_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn2_macro_nonfused_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,16,16,2,2,2,2,256,99
1,32,8,4,2,2,2,256,70
2,64,4,8,2,2,2,256,35
3,128,2,16,2,2,2,256,17
4,248,1,31,2,2,2,248,4
5,256,1,32,2,2,2,256,7


### Cases Setup 

In [71]:
dfn2MacNonFusProb_16mpi_16omp = dfn2_macro_nonfused_probempi[(dfn2_macro_nonfused_probempi.omp_thrd == 16) & (dfn2_macro_nonfused_probempi.GLB_T/dfn2_macro_nonfused_probempi.LOC_T == 2) & (dfn2_macro_nonfused_probempi.GLB_X/dfn2_macro_nonfused_probempi.LOC_X == 2) & (dfn2_macro_nonfused_probempi.GLB_Y/dfn2_macro_nonfused_probempi.LOC_Y == 2)& (dfn2_macro_nonfused_probempi.GLB_Z/dfn2_macro_nonfused_probempi.LOC_Z == 2)]
dfn2MacNonFusProb_32mpi_8omp = dfn2_macro_nonfused_probempi[(dfn2_macro_nonfused_probempi.omp_thrd == 8) & (dfn2_macro_nonfused_probempi.GLB_T/dfn2_macro_nonfused_probempi.LOC_T == 4) & (dfn2_macro_nonfused_probempi.GLB_X/dfn2_macro_nonfused_probempi.LOC_X == 2) & (dfn2_macro_nonfused_probempi.GLB_Y/dfn2_macro_nonfused_probempi.LOC_Y == 2)& (dfn2_macro_nonfused_probempi.GLB_Z/dfn2_macro_nonfused_probempi.LOC_Z == 2)]
dfn2MacNonFusProb_64mpi_4omp = dfn2_macro_nonfused_probempi[(dfn2_macro_nonfused_probempi.omp_thrd == 4) & (dfn2_macro_nonfused_probempi.GLB_T/dfn2_macro_nonfused_probempi.LOC_T == 8) & (dfn2_macro_nonfused_probempi.GLB_X/dfn2_macro_nonfused_probempi.LOC_X == 2) & (dfn2_macro_nonfused_probempi.GLB_Y/dfn2_macro_nonfused_probempi.LOC_Y == 2)& (dfn2_macro_nonfused_probempi.GLB_Z/dfn2_macro_nonfused_probempi.LOC_Z == 2)]
dfn2MacNonFusProb_128mpi_2omp = dfn2_macro_nonfused_probempi[(dfn2_macro_nonfused_probempi.omp_thrd == 2) & (dfn2_macro_nonfused_probempi.GLB_T/dfn2_macro_nonfused_probempi.LOC_T == 16) & (dfn2_macro_nonfused_probempi.GLB_X/dfn2_macro_nonfused_probempi.LOC_X == 2) & (dfn2_macro_nonfused_probempi.GLB_Y/dfn2_macro_nonfused_probempi.LOC_Y == 2)& (dfn2_macro_nonfused_probempi.GLB_Z/dfn2_macro_nonfused_probempi.LOC_Z == 2)]
dfn2MacNonFusProb_256mpi_1omp = dfn2_macro_nonfused_probempi[(dfn2_macro_nonfused_probempi.omp_thrd == 1) & (dfn2_macro_nonfused_probempi.GLB_T/dfn2_macro_nonfused_probempi.LOC_T == 32) & (dfn2_macro_nonfused_probempi.GLB_X/dfn2_macro_nonfused_probempi.LOC_X == 2) & (dfn2_macro_nonfused_probempi.GLB_Y/dfn2_macro_nonfused_probempi.LOC_Y == 2)& (dfn2_macro_nonfused_probempi.GLB_Z/dfn2_macro_nonfused_probempi.LOC_Z == 2)]


## 4) Nonfused_Macro_NonProbempi

### Adding Columns

In [72]:
dfn2_macro_nonfused_nonprobempi["N_Nodes"] = 2
dfn2_macro_nonfused_nonprobempi['NPX'] = (dfn2_macro_nonfused_nonprobempi.GLB_X//dfn2_macro_nonfused_nonprobempi.LOC_X)
dfn2_macro_nonfused_nonprobempi['NPY'] = (dfn2_macro_nonfused_nonprobempi.GLB_Y//dfn2_macro_nonfused_nonprobempi.LOC_Y)
dfn2_macro_nonfused_nonprobempi['NPZ'] = (dfn2_macro_nonfused_nonprobempi.GLB_Z//dfn2_macro_nonfused_nonprobempi.LOC_Z)
dfn2_macro_nonfused_nonprobempi['NPT'] = (dfn2_macro_nonfused_nonprobempi.GLB_T//dfn2_macro_nonfused_nonprobempi.LOC_T)
dfn2_macro_nonfused_nonprobempi["Total_proc"] = dfn2_macro_nonfused_nonprobempi["omp_thrd"]* dfn2_macro_nonfused_nonprobempi.MPI_size
# dfn2_macro_nonfused_nonprobempi.columns

### Data Summary

In [73]:
dfn2_macro_nonfused_nonprobempi_summary = dfn2_macro_nonfused_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn2_macro_nonfused_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,16,16,2,2,2,2,256,104
1,32,8,4,2,2,2,256,69
2,64,4,8,2,2,2,256,35
3,128,2,16,2,2,2,256,17
4,248,1,31,2,2,2,248,4
5,256,1,32,2,2,2,256,6


### Cases Setup 

In [74]:
dfn2MacNonFusNonProb_16mpi_16omp = dfn2_macro_nonfused_nonprobempi[(dfn2_macro_nonfused_nonprobempi.omp_thrd == 16) & (dfn2_macro_nonfused_nonprobempi.GLB_T/dfn2_macro_nonfused_nonprobempi.LOC_T == 2) & (dfn2_macro_nonfused_nonprobempi.GLB_X/dfn2_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn2_macro_nonfused_nonprobempi.GLB_Y/dfn2_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn2_macro_nonfused_nonprobempi.GLB_Z/dfn2_macro_nonfused_nonprobempi.LOC_Z == 2)]
dfn2MacNonFusNonProb_32mpi_8omp = dfn2_macro_nonfused_nonprobempi[(dfn2_macro_nonfused_nonprobempi.omp_thrd == 8) & (dfn2_macro_nonfused_nonprobempi.GLB_T/dfn2_macro_nonfused_nonprobempi.LOC_T == 4) & (dfn2_macro_nonfused_nonprobempi.GLB_X/dfn2_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn2_macro_nonfused_nonprobempi.GLB_Y/dfn2_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn2_macro_nonfused_nonprobempi.GLB_Z/dfn2_macro_nonfused_nonprobempi.LOC_Z == 2)]
dfn2MacNonFusNonProb_64mpi_4omp = dfn2_macro_nonfused_nonprobempi[(dfn2_macro_nonfused_nonprobempi.omp_thrd == 4) & (dfn2_macro_nonfused_nonprobempi.GLB_T/dfn2_macro_nonfused_nonprobempi.LOC_T == 8) & (dfn2_macro_nonfused_nonprobempi.GLB_X/dfn2_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn2_macro_nonfused_nonprobempi.GLB_Y/dfn2_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn2_macro_nonfused_nonprobempi.GLB_Z/dfn2_macro_nonfused_nonprobempi.LOC_Z == 2)]
dfn2MacNonFusNonProb_128mpi_2omp = dfn2_macro_nonfused_nonprobempi[(dfn2_macro_nonfused_nonprobempi.omp_thrd == 2) & (dfn2_macro_nonfused_nonprobempi.GLB_T/dfn2_macro_nonfused_nonprobempi.LOC_T == 16) & (dfn2_macro_nonfused_nonprobempi.GLB_X/dfn2_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn2_macro_nonfused_nonprobempi.GLB_Y/dfn2_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn2_macro_nonfused_nonprobempi.GLB_Z/dfn2_macro_nonfused_nonprobempi.LOC_Z == 2)]
dfn2MacNonFusNonProb_256mpi_1omp = dfn2_macro_nonfused_nonprobempi[(dfn2_macro_nonfused_nonprobempi.omp_thrd == 1) & (dfn2_macro_nonfused_nonprobempi.GLB_T/dfn2_macro_nonfused_nonprobempi.LOC_T == 32) & (dfn2_macro_nonfused_nonprobempi.GLB_X/dfn2_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn2_macro_nonfused_nonprobempi.GLB_Y/dfn2_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn2_macro_nonfused_nonprobempi.GLB_Z/dfn2_macro_nonfused_nonprobempi.LOC_Z == 2)]


## GFLOPS - Nonfused Macro: Probempi vs Nonprobempi

In [75]:
fig = go.Figure() 

fig.add_trace(go.Scatter(x=(dfn2MacNonFusNonProb_256mpi_1omp.Dirc_Data_Mov*(dfn2MacNonFusNonProb_256mpi_1omp.MPI_size/2)), 
                         y=(dfn2MacNonFusNonProb_256mpi_1omp.GFLOPS*(dfn2MacNonFusNonProb_256mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_256mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


fig.add_trace(go.Scatter(x=(dfn2MacNonFusProb_256mpi_1omp.Dirc_Data_Mov*(dfn2MacNonFusProb_256mpi_1omp.MPI_size/2)), 
                         y=(dfn2MacNonFusProb_256mpi_1omp.GFLOPS*(dfn2MacNonFusProb_256mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_256mpi_1omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

# case 2
fig.add_trace(go.Scatter(x=(dfn2MacNonFusNonProb_128mpi_2omp.Dirc_Data_Mov*(dfn2MacNonFusNonProb_128mpi_2omp.MPI_size/2)), 
                         y=(dfn2MacNonFusNonProb_128mpi_2omp.GFLOPS*(dfn2MacNonFusNonProb_128mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_128mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn2MacNonFusProb_128mpi_2omp.Dirc_Data_Mov*(dfn2MacNonFusProb_128mpi_2omp.MPI_size/2)), 
                         y=(dfn2MacNonFusProb_128mpi_2omp.GFLOPS*(dfn2MacNonFusProb_128mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_128mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

# case 3
fig.add_trace(go.Scatter(x=(dfn2MacNonFusNonProb_64mpi_4omp.Dirc_Data_Mov*(dfn2MacNonFusNonProb_64mpi_4omp.MPI_size/2)), 
                         y=(dfn2MacNonFusNonProb_64mpi_4omp.GFLOPS*(dfn2MacNonFusNonProb_64mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_64mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))

fig.add_trace(go.Scatter(x=(dfn2MacNonFusProb_64mpi_4omp.Dirc_Data_Mov*(dfn2MacNonFusProb_64mpi_4omp.MPI_size/2)), 
                         y=(dfn2MacNonFusProb_64mpi_4omp.GFLOPS*(dfn2MacNonFusProb_64mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_64mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

# case 4
fig.add_trace(go.Scatter(x=(dfn2MacNonFusNonProb_32mpi_8omp.Dirc_Data_Mov*(dfn2MacNonFusNonProb_32mpi_8omp.MPI_size/2)), 
                         y=(dfn2MacNonFusNonProb_32mpi_8omp.GFLOPS*(dfn2MacNonFusNonProb_32mpi_8omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_32mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='red',width=2))))

fig.add_trace(go.Scatter(x=(dfn2MacNonFusProb_32mpi_8omp.Dirc_Data_Mov*(dfn2MacNonFusProb_32mpi_8omp.MPI_size/2)), 
                         y=(dfn2MacNonFusProb_32mpi_8omp.GFLOPS*(dfn2MacNonFusProb_32mpi_8omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_32mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='green',width=2))))

# case 5

fig.add_trace(go.Scatter(x=(dfn2MacNonFusNonProb_16mpi_16omp.Dirc_Data_Mov*(dfn2MacNonFusNonProb_16mpi_16omp.MPI_size/2)), 
                         y=(dfn2MacNonFusNonProb_16mpi_16omp.GFLOPS*(dfn2MacNonFusNonProb_16mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_16mpi_16omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='chocolate',width=2))))

fig.add_trace(go.Scatter(x=(dfn2MacNonFusProb_16mpi_16omp.Dirc_Data_Mov*(dfn2MacNonFusProb_16mpi_16omp.MPI_size/2)), 
                         y=(dfn2MacNonFusProb_16mpi_16omp.GFLOPS*(dfn2MacNonFusProb_16mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_16mpi_16omp',
                         line=dict(color='darkviolet',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='crimson',width=2))))



# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"NonFused-Macro (2-Nodes): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1))

## 5) Newgeo_Macro_Probempi

### Adding Columns

In [76]:
dfn2_macro_newgeo_probempi["N_Nodes"] = 2
dfn2_macro_newgeo_probempi['NPX'] = (dfn2_macro_newgeo_probempi.GLB_X//dfn2_macro_newgeo_probempi.LOC_X)
dfn2_macro_newgeo_probempi['NPY'] = (dfn2_macro_newgeo_probempi.GLB_Y//dfn2_macro_newgeo_probempi.LOC_Y)
dfn2_macro_newgeo_probempi['NPZ'] = (dfn2_macro_newgeo_probempi.GLB_Z//dfn2_macro_newgeo_probempi.LOC_Z)
dfn2_macro_newgeo_probempi['NPT'] = (dfn2_macro_newgeo_probempi.GLB_T//dfn2_macro_newgeo_probempi.LOC_T)
dfn2_macro_newgeo_probempi["Total_proc"] = dfn2_macro_newgeo_probempi["omp_thrd"]* dfn2_macro_newgeo_probempi.MPI_size
# dfn2_macro_newgeo_probempi.columns

### Data Summary

In [77]:
dfn2_macro_newgeo_probempi_summary =dfn2_macro_newgeo_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn2_macro_newgeo_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,16,16,2,2,2,2,256,140
1,32,8,4,2,2,2,256,69
2,64,4,8,2,2,2,256,35
3,128,2,16,2,2,2,256,17
4,248,1,31,2,2,2,248,4
5,256,1,32,2,2,2,256,7


### Cases Setup 

In [78]:
dfn2MacNewgeoProb_16mpi_16omp = dfn2_macro_newgeo_probempi[(dfn2_macro_newgeo_probempi.omp_thrd == 16) & (dfn2_macro_newgeo_probempi.GLB_T/dfn2_macro_newgeo_probempi.LOC_T == 2) & (dfn2_macro_newgeo_probempi.GLB_X/dfn2_macro_newgeo_probempi.LOC_X == 2) & (dfn2_macro_newgeo_probempi.GLB_Y/dfn2_macro_newgeo_probempi.LOC_Y == 2)& (dfn2_macro_newgeo_probempi.GLB_Z/dfn2_macro_newgeo_probempi.LOC_Z == 2)]
dfn2MacNewgeoProb_32mpi_8omp = dfn2_macro_newgeo_probempi[(dfn2_macro_newgeo_probempi.omp_thrd == 8) & (dfn2_macro_newgeo_probempi.GLB_T/dfn2_macro_newgeo_probempi.LOC_T == 4) & (dfn2_macro_newgeo_probempi.GLB_X/dfn2_macro_newgeo_probempi.LOC_X == 2) & (dfn2_macro_newgeo_probempi.GLB_Y/dfn2_macro_newgeo_probempi.LOC_Y == 2)& (dfn2_macro_newgeo_probempi.GLB_Z/dfn2_macro_newgeo_probempi.LOC_Z == 2)]
dfn2MacNewgeoProb_64mpi_4omp = dfn2_macro_newgeo_probempi[(dfn2_macro_newgeo_probempi.omp_thrd == 4) & (dfn2_macro_newgeo_probempi.GLB_T/dfn2_macro_newgeo_probempi.LOC_T == 8) & (dfn2_macro_newgeo_probempi.GLB_X/dfn2_macro_newgeo_probempi.LOC_X == 2) & (dfn2_macro_newgeo_probempi.GLB_Y/dfn2_macro_newgeo_probempi.LOC_Y == 2)& (dfn2_macro_newgeo_probempi.GLB_Z/dfn2_macro_newgeo_probempi.LOC_Z == 2)]
dfn2MacNewgeoProb_128mpi_2omp = dfn2_macro_newgeo_probempi[(dfn2_macro_newgeo_probempi.omp_thrd == 2) & (dfn2_macro_newgeo_probempi.GLB_T/dfn2_macro_newgeo_probempi.LOC_T == 16) & (dfn2_macro_newgeo_probempi.GLB_X/dfn2_macro_newgeo_probempi.LOC_X == 2) & (dfn2_macro_newgeo_probempi.GLB_Y/dfn2_macro_newgeo_probempi.LOC_Y == 2)& (dfn2_macro_newgeo_probempi.GLB_Z/dfn2_macro_newgeo_probempi.LOC_Z == 2)]
dfn2MacNewgeoProb_256mpi_1omp = dfn2_macro_newgeo_probempi[(dfn2_macro_newgeo_probempi.omp_thrd == 1) & (dfn2_macro_newgeo_probempi.GLB_T/dfn2_macro_newgeo_probempi.LOC_T == 32) & (dfn2_macro_newgeo_probempi.GLB_X/dfn2_macro_newgeo_probempi.LOC_X == 2) & (dfn2_macro_newgeo_probempi.GLB_Y/dfn2_macro_newgeo_probempi.LOC_Y == 2)& (dfn2_macro_newgeo_probempi.GLB_Z/dfn2_macro_newgeo_probempi.LOC_Z == 2)]


## 6) Newgeo_Macro_NonProbempi

### Adding Columns

In [86]:
dfn2_macro_newgeo_nonprobempi["N_Nodes"] = 2
dfn2_macro_newgeo_nonprobempi['NPX'] = (dfn2_macro_newgeo_nonprobempi.GLB_X//dfn2_macro_newgeo_nonprobempi.LOC_X)
dfn2_macro_newgeo_nonprobempi['NPY'] = (dfn2_macro_newgeo_nonprobempi.GLB_Y//dfn2_macro_newgeo_nonprobempi.LOC_Y)
dfn2_macro_newgeo_nonprobempi['NPZ'] = (dfn2_macro_newgeo_nonprobempi.GLB_Z//dfn2_macro_newgeo_nonprobempi.LOC_Z)
dfn2_macro_newgeo_nonprobempi['NPT'] = (dfn2_macro_newgeo_nonprobempi.GLB_T//dfn2_macro_newgeo_nonprobempi.LOC_T)
dfn2_macro_newgeo_nonprobempi["Total_proc"] = dfn2_macro_newgeo_nonprobempi["omp_thrd"]* dfn2_macro_newgeo_nonprobempi.MPI_size
# dfn2_macro_newgeo_nonprobempi.columns

### Data Summary

In [87]:
dfn2_macro_newgeo_nonprobempi_summary = dfn2_macro_newgeo_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn2_macro_newgeo_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,16,16,2,2,2,2,256,139
1,32,8,4,2,2,2,256,70
2,64,4,8,2,2,2,256,34
3,128,2,16,2,2,2,256,17
4,248,1,31,2,2,2,248,4
5,256,1,32,2,2,2,256,7


### Cases Setup 

In [94]:
dfn2MacNewgeoNonProb_16mpi_16omp = dfn2_macro_newgeo_nonprobempi[(dfn2_macro_newgeo_nonprobempi.omp_thrd == 16) & (dfn2_macro_newgeo_nonprobempi.GLB_T/dfn2_macro_newgeo_nonprobempi.LOC_T == 2) & (dfn2_macro_newgeo_nonprobempi.GLB_X/dfn2_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn2_macro_newgeo_nonprobempi.GLB_Y/dfn2_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn2_macro_newgeo_nonprobempi.GLB_Z/dfn2_macro_newgeo_nonprobempi.LOC_Z == 2)]
dfn2MacNewgeoNonProb_32mpi_8omp = dfn2_macro_newgeo_nonprobempi[(dfn2_macro_newgeo_nonprobempi.omp_thrd == 8) & (dfn2_macro_newgeo_nonprobempi.GLB_T/dfn2_macro_newgeo_nonprobempi.LOC_T == 4) & (dfn2_macro_newgeo_nonprobempi.GLB_X/dfn2_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn2_macro_newgeo_nonprobempi.GLB_Y/dfn2_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn2_macro_newgeo_nonprobempi.GLB_Z/dfn2_macro_newgeo_nonprobempi.LOC_Z == 2)]
dfn2MacNewgeoNonProb_64mpi_4omp = dfn2_macro_newgeo_nonprobempi[(dfn2_macro_newgeo_nonprobempi.omp_thrd == 4) & (dfn2_macro_newgeo_nonprobempi.GLB_T/dfn2_macro_newgeo_nonprobempi.LOC_T == 8) & (dfn2_macro_newgeo_nonprobempi.GLB_X/dfn2_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn2_macro_newgeo_nonprobempi.GLB_Y/dfn2_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn2_macro_newgeo_nonprobempi.GLB_Z/dfn2_macro_newgeo_nonprobempi.LOC_Z == 2)]
dfn2MacNewgeoNonProb_128mpi_2omp = dfn2_macro_newgeo_nonprobempi[(dfn2_macro_newgeo_nonprobempi.omp_thrd == 2) & (dfn2_macro_newgeo_nonprobempi.GLB_T/dfn2_macro_newgeo_nonprobempi.LOC_T == 16) & (dfn2_macro_newgeo_nonprobempi.GLB_X/dfn2_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn2_macro_newgeo_nonprobempi.GLB_Y/dfn2_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn2_macro_newgeo_nonprobempi.GLB_Z/dfn2_macro_newgeo_nonprobempi.LOC_Z == 2)]
dfn2MacNewgeoNonProb_256mpi_1omp = dfn2_macro_newgeo_nonprobempi[(dfn2_macro_newgeo_nonprobempi.omp_thrd == 1) & (dfn2_macro_newgeo_nonprobempi.GLB_T/dfn2_macro_newgeo_nonprobempi.LOC_T == 32) & (dfn2_macro_newgeo_nonprobempi.GLB_X/dfn2_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn2_macro_newgeo_nonprobempi.GLB_Y/dfn2_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn2_macro_newgeo_nonprobempi.GLB_Z/dfn2_macro_newgeo_nonprobempi.LOC_Z == 2)]


## GFLOPS - Newgeo Macro: Probempi vs Nonprobempi

In [95]:
fig = go.Figure() 

fig.add_trace(go.Scatter(x=(dfn2MacNewgeoNonProb_256mpi_1omp.Dirc_Data_Mov*(dfn2MacNewgeoNonProb_256mpi_1omp.MPI_size/2)), 
                         y=(dfn2MacNewgeoNonProb_256mpi_1omp.GFLOPS*(dfn2MacNewgeoNonProb_256mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_256mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


fig.add_trace(go.Scatter(x=(dfn2MacNewgeoProb_256mpi_1omp.Dirc_Data_Mov*(dfn2MacNewgeoProb_256mpi_1omp.MPI_size/2)), 
                         y=(dfn2MacNewgeoProb_256mpi_1omp.GFLOPS*(dfn2MacNewgeoProb_256mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_256mpi_1omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

# case 2
fig.add_trace(go.Scatter(x=(dfn2MacNewgeoNonProb_128mpi_2omp.Dirc_Data_Mov*(dfn2MacNewgeoNonProb_128mpi_2omp.MPI_size/2)), 
                         y=(dfn2MacNewgeoNonProb_128mpi_2omp.GFLOPS*(dfn2MacNewgeoNonProb_128mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_128mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn2MacNewgeoProb_128mpi_2omp.Dirc_Data_Mov*(dfn2MacNewgeoProb_128mpi_2omp.MPI_size/2)), 
                         y=(dfn2MacNewgeoProb_128mpi_2omp.GFLOPS*(dfn2MacNewgeoProb_128mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_128mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

# case 3
fig.add_trace(go.Scatter(x=(dfn2MacNewgeoNonProb_64mpi_4omp.Dirc_Data_Mov*(dfn2MacNewgeoNonProb_64mpi_4omp.MPI_size/2)), 
                         y=(dfn2MacNewgeoNonProb_64mpi_4omp.GFLOPS*(dfn2MacNewgeoNonProb_64mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_64mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))

fig.add_trace(go.Scatter(x=(dfn2MacNewgeoProb_64mpi_4omp.Dirc_Data_Mov*(dfn2MacNewgeoProb_64mpi_4omp.MPI_size/2)), 
                         y=(dfn2MacNewgeoProb_64mpi_4omp.GFLOPS*(dfn2MacNewgeoProb_64mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_64mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

# case 4
fig.add_trace(go.Scatter(x=(dfn2MacNewgeoNonProb_32mpi_8omp.Dirc_Data_Mov*(dfn2MacNewgeoNonProb_32mpi_8omp.MPI_size/2)), 
                         y=(dfn2MacNewgeoNonProb_32mpi_8omp.GFLOPS*(dfn2MacNewgeoNonProb_32mpi_8omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_32mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='red',width=2))))

fig.add_trace(go.Scatter(x=(dfn2MacNewgeoProb_32mpi_8omp.Dirc_Data_Mov*(dfn2MacNewgeoProb_32mpi_8omp.MPI_size/2)), 
                         y=(dfn2MacNewgeoProb_32mpi_8omp.GFLOPS*(dfn2MacNewgeoProb_32mpi_8omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_32mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='green',width=2))))

# case 5

fig.add_trace(go.Scatter(x=(dfn2MacNewgeoNonProb_16mpi_16omp.Dirc_Data_Mov*(dfn2MacNewgeoNonProb_16mpi_16omp.MPI_size/2)), 
                         y=(dfn2MacNewgeoNonProb_16mpi_16omp.GFLOPS*(dfn2MacNewgeoNonProb_16mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_16mpi_16omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='chocolate',width=2))))

fig.add_trace(go.Scatter(x=(dfn2MacNewgeoProb_16mpi_16omp.Dirc_Data_Mov*(dfn2MacNewgeoProb_16mpi_16omp.MPI_size/2)), 
                         y=(dfn2MacNewgeoProb_16mpi_16omp.GFLOPS*(dfn2MacNewgeoProb_16mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_16mpi_16omp',
                         line=dict(color='darkviolet',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='crimson',width=2))))



# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"Newgeo-Macro (2-Nodes): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1))

# HIPPO AVX CASES

## 1) Fused_AVX_Probmpi

### Adding Columns

In [98]:
dfn2_avx_fused_probmpi["N_Nodes"] = 2
dfn2_avx_fused_probmpi['NPX'] = (dfn2_avx_fused_probmpi.GLB_X//dfn2_avx_fused_probmpi.LOC_X)
dfn2_avx_fused_probmpi['NPY'] = (dfn2_avx_fused_probmpi.GLB_Y//dfn2_avx_fused_probmpi.LOC_Y)
dfn2_avx_fused_probmpi['NPZ'] = (dfn2_avx_fused_probmpi.GLB_Z//dfn2_avx_fused_probmpi.LOC_Z)
dfn2_avx_fused_probmpi['NPT'] = (dfn2_avx_fused_probmpi.GLB_T//dfn2_avx_fused_probmpi.LOC_T)
dfn2_avx_fused_probmpi["Total_proc"] = dfn2_avx_fused_probmpi["omp_thrd"]* dfn2_avx_fused_probmpi.MPI_size
# dfn2_avx_fused_probmpi.columns

### Data Summary

In [99]:
dfn2_avx_fused_probmpi_summary = dfn2_avx_fused_probmpi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn2_avx_fused_probmpi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,16,16,2,2,2,2,256,139
1,32,8,4,2,2,2,256,68
2,64,4,8,2,2,2,256,35
3,128,2,16,2,2,2,256,17
4,248,1,31,2,2,2,248,4
5,256,1,32,2,2,2,256,6


### Cases Setup 

In [105]:
dfn2AvxFusProb_16mpi_16omp = dfn2_avx_fused_probmpi[(dfn2_avx_fused_probmpi.omp_thrd == 16) & (dfn2_avx_fused_probmpi.GLB_T/dfn2_avx_fused_probmpi.LOC_T == 2) & (dfn2_avx_fused_probmpi.GLB_X/dfn2_avx_fused_probmpi.LOC_X == 2) & (dfn2_avx_fused_probmpi.GLB_Y/dfn2_avx_fused_probmpi.LOC_Y == 2)& (dfn2_avx_fused_probmpi.GLB_Z/dfn2_avx_fused_probmpi.LOC_Z == 2)]
dfn2AvxFusProb_32mpi_8omp = dfn2_avx_fused_probmpi[(dfn2_avx_fused_probmpi.omp_thrd == 8) & (dfn2_avx_fused_probmpi.GLB_T/dfn2_avx_fused_probmpi.LOC_T == 4) & (dfn2_avx_fused_probmpi.GLB_X/dfn2_avx_fused_probmpi.LOC_X == 2) & (dfn2_avx_fused_probmpi.GLB_Y/dfn2_avx_fused_probmpi.LOC_Y == 2)& (dfn2_avx_fused_probmpi.GLB_Z/dfn2_avx_fused_probmpi.LOC_Z == 2)]
dfn2AvxFusProb_64mpi_4omp = dfn2_avx_fused_probmpi[(dfn2_avx_fused_probmpi.omp_thrd == 4) & (dfn2_avx_fused_probmpi.GLB_T/dfn2_avx_fused_probmpi.LOC_T == 8) & (dfn2_avx_fused_probmpi.GLB_X/dfn2_avx_fused_probmpi.LOC_X == 2) & (dfn2_avx_fused_probmpi.GLB_Y/dfn2_avx_fused_probmpi.LOC_Y == 2)& (dfn2_avx_fused_probmpi.GLB_Z/dfn2_avx_fused_probmpi.LOC_Z == 2)]
dfn2AvxFusProb_128mpi_2omp = dfn2_avx_fused_probmpi[(dfn2_avx_fused_probmpi.omp_thrd == 2) & (dfn2_avx_fused_probmpi.GLB_T/dfn2_avx_fused_probmpi.LOC_T == 16) & (dfn2_avx_fused_probmpi.GLB_X/dfn2_avx_fused_probmpi.LOC_X == 2) & (dfn2_avx_fused_probmpi.GLB_Y/dfn2_avx_fused_probmpi.LOC_Y == 2)& (dfn2_avx_fused_probmpi.GLB_Z/dfn2_avx_fused_probmpi.LOC_Z == 2)]
dfn2AvxFusProb_256mpi_1omp = dfn2_avx_fused_probmpi[(dfn2_avx_fused_probmpi.omp_thrd == 1) & (dfn2_avx_fused_probmpi.GLB_T/dfn2_avx_fused_probmpi.LOC_T == 32) & (dfn2_avx_fused_probmpi.GLB_X/dfn2_avx_fused_probmpi.LOC_X == 2) & (dfn2_avx_fused_probmpi.GLB_Y/dfn2_avx_fused_probmpi.LOC_Y == 2)& (dfn2_avx_fused_probmpi.GLB_Z/dfn2_avx_fused_probmpi.LOC_Z == 2)]


## 2) Fused_AVX_NonProbmpi

## Adding Columns

In [106]:
dfn2_avx_fused_nonprobmpi["N_Nodes"] = 2
dfn2_avx_fused_nonprobmpi['NPX'] = (dfn2_avx_fused_nonprobmpi.GLB_X//dfn2_avx_fused_nonprobmpi.LOC_X)
dfn2_avx_fused_nonprobmpi['NPY'] = (dfn2_avx_fused_nonprobmpi.GLB_Y//dfn2_avx_fused_nonprobmpi.LOC_Y)
dfn2_avx_fused_nonprobmpi['NPZ'] = (dfn2_avx_fused_nonprobmpi.GLB_Z//dfn2_avx_fused_nonprobmpi.LOC_Z)
dfn2_avx_fused_nonprobmpi['NPT'] = (dfn2_avx_fused_nonprobmpi.GLB_T//dfn2_avx_fused_nonprobmpi.LOC_T)
dfn2_avx_fused_nonprobmpi["Total_proc"] = dfn2_avx_fused_nonprobmpi["omp_thrd"]* dfn2_avx_fused_nonprobmpi.MPI_size
# dfn2_avx_fused_nonprobmpi.columns

### Data Summary

In [107]:
dfn2_avx_fused_nonprobmpi_summary = dfn2_avx_fused_nonprobmpi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn2_avx_fused_nonprobmpi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,16,16,2,2,2,2,256,139
1,32,8,4,2,2,2,256,70
2,64,4,8,2,2,2,256,35
3,128,2,16,2,2,2,256,17
4,248,1,31,2,2,2,248,4
5,256,1,32,2,2,2,256,7


### Cases Setup

In [113]:
dfn2AvxFusNonProb_16mpi_16omp = dfn2_avx_fused_nonprobmpi[(dfn2_avx_fused_nonprobmpi.omp_thrd == 16) & (dfn2_avx_fused_nonprobmpi.GLB_T/dfn2_avx_fused_nonprobmpi.LOC_T == 2) & (dfn2_avx_fused_nonprobmpi.GLB_X/dfn2_avx_fused_nonprobmpi.LOC_X == 2) & (dfn2_avx_fused_nonprobmpi.GLB_Y/dfn2_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn2_avx_fused_nonprobmpi.GLB_Z/dfn2_avx_fused_nonprobmpi.LOC_Z == 2)]
dfn2AvxFusNonProb_32mpi_8omp = dfn2_avx_fused_nonprobmpi[(dfn2_avx_fused_nonprobmpi.omp_thrd == 8) & (dfn2_avx_fused_nonprobmpi.GLB_T/dfn2_avx_fused_nonprobmpi.LOC_T == 4) & (dfn2_avx_fused_nonprobmpi.GLB_X/dfn2_avx_fused_nonprobmpi.LOC_X == 2) & (dfn2_avx_fused_nonprobmpi.GLB_Y/dfn2_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn2_avx_fused_nonprobmpi.GLB_Z/dfn2_avx_fused_nonprobmpi.LOC_Z == 2)]
dfn2AvxFusNonProb_64mpi_4omp = dfn2_avx_fused_nonprobmpi[(dfn2_avx_fused_nonprobmpi.omp_thrd == 4) & (dfn2_avx_fused_nonprobmpi.GLB_T/dfn2_avx_fused_nonprobmpi.LOC_T == 8) & (dfn2_avx_fused_nonprobmpi.GLB_X/dfn2_avx_fused_nonprobmpi.LOC_X == 2) & (dfn2_avx_fused_nonprobmpi.GLB_Y/dfn2_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn2_avx_fused_nonprobmpi.GLB_Z/dfn2_avx_fused_nonprobmpi.LOC_Z == 2)]
dfn2AvxFusNonProb_128mpi_2omp = dfn2_avx_fused_nonprobmpi[(dfn2_avx_fused_nonprobmpi.omp_thrd == 2) & (dfn2_avx_fused_nonprobmpi.GLB_T/dfn2_avx_fused_nonprobmpi.LOC_T == 16) & (dfn2_avx_fused_nonprobmpi.GLB_X/dfn2_avx_fused_nonprobmpi.LOC_X == 2) & (dfn2_avx_fused_nonprobmpi.GLB_Y/dfn2_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn2_avx_fused_nonprobmpi.GLB_Z/dfn2_avx_fused_nonprobmpi.LOC_Z == 2)]
dfn2AvxFusNonProb_256mpi_1omp = dfn2_avx_fused_nonprobmpi[(dfn2_avx_fused_nonprobmpi.omp_thrd == 1) & (dfn2_avx_fused_nonprobmpi.GLB_T/dfn2_avx_fused_nonprobmpi.LOC_T == 32) & (dfn2_avx_fused_nonprobmpi.GLB_X/dfn2_avx_fused_nonprobmpi.LOC_X == 2) & (dfn2_avx_fused_nonprobmpi.GLB_Y/dfn2_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn2_avx_fused_nonprobmpi.GLB_Z/dfn2_avx_fused_nonprobmpi.LOC_Z == 2)]


## GFLOPS - AVX-Fused: Probmpi vs Non-Probempim

In [114]:
fig = go.Figure() 

fig.add_trace(go.Scatter(x=(dfn2AvxFusNonProb_256mpi_1omp.Dirc_Data_Mov*(dfn2AvxFusNonProb_256mpi_1omp.MPI_size/2)), 
                         y=(dfn2AvxFusNonProb_256mpi_1omp.GFLOPS*(dfn2AvxFusNonProb_256mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_256mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


fig.add_trace(go.Scatter(x=(dfn2AvxFusProb_256mpi_1omp.Dirc_Data_Mov*(dfn2AvxFusProb_256mpi_1omp.MPI_size/2)), 
                         y=(dfn2AvxFusProb_256mpi_1omp.GFLOPS*(dfn2AvxFusProb_256mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxFusProb_256mpi_1omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

# case 2
fig.add_trace(go.Scatter(x=(dfn2AvxFusNonProb_128mpi_2omp.Dirc_Data_Mov*(dfn2AvxFusNonProb_128mpi_2omp.MPI_size/2)), 
                         y=(dfn2AvxFusNonProb_128mpi_2omp.GFLOPS*(dfn2AvxFusNonProb_128mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_128mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn2AvxFusProb_128mpi_2omp.Dirc_Data_Mov*(dfn2AvxFusProb_128mpi_2omp.MPI_size/2)), 
                         y=(dfn2AvxFusProb_128mpi_2omp.GFLOPS*(dfn2AvxFusProb_128mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxFusProb_128mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

# case 3
fig.add_trace(go.Scatter(x=(dfn2AvxFusNonProb_64mpi_4omp.Dirc_Data_Mov*(dfn2AvxFusNonProb_64mpi_4omp.MPI_size/2)), 
                         y=(dfn2AvxFusNonProb_64mpi_4omp.GFLOPS*(dfn2AvxFusNonProb_64mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_64mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))

fig.add_trace(go.Scatter(x=(dfn2AvxFusProb_64mpi_4omp.Dirc_Data_Mov*(dfn2AvxFusProb_64mpi_4omp.MPI_size/2)), 
                         y=(dfn2AvxFusProb_64mpi_4omp.GFLOPS*(dfn2AvxFusProb_64mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxFusProb_64mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

# case 4
fig.add_trace(go.Scatter(x=(dfn2AvxFusNonProb_32mpi_8omp.Dirc_Data_Mov*(dfn2AvxFusNonProb_32mpi_8omp.MPI_size/2)), 
                         y=(dfn2AvxFusNonProb_32mpi_8omp.GFLOPS*(dfn2AvxFusNonProb_32mpi_8omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_32mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='red',width=2))))

fig.add_trace(go.Scatter(x=(dfn2AvxFusProb_32mpi_8omp.Dirc_Data_Mov*(dfn2AvxFusProb_32mpi_8omp.MPI_size/2)), 
                         y=(dfn2AvxFusProb_32mpi_8omp.GFLOPS*(dfn2AvxFusProb_32mpi_8omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='dfn2AvxFusProb_32mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='green',width=2))))

# case 5

fig.add_trace(go.Scatter(x=(dfn2AvxFusNonProb_16mpi_16omp.Dirc_Data_Mov*(dfn2AvxFusNonProb_16mpi_16omp.MPI_size/2)), 
                         y=(dfn2AvxFusNonProb_16mpi_16omp.GFLOPS*(dfn2AvxFusNonProb_16mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_16mpi_16omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='chocolate',width=2))))

fig.add_trace(go.Scatter(x=(dfn2AvxFusProb_16mpi_16omp.Dirc_Data_Mov*(dfn2AvxFusProb_16mpi_16omp.MPI_size/2)), 
                         y=(dfn2AvxFusProb_16mpi_16omp.GFLOPS*(dfn2AvxFusProb_16mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='dfn2AvxFusProb_16mpi_16omp',
                         line=dict(color='darkviolet',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='crimson',width=2))))



# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"Fused-AVX (2-Nodes): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1))

## 3) NonFused_AVX_Probempi

### Adding Columns

In [115]:
dfn2_avx_nonfused_probempi["N_Nodes"] = 2
dfn2_avx_nonfused_probempi['NPX'] = (dfn2_avx_nonfused_probempi.GLB_X//dfn2_avx_nonfused_probempi.LOC_X)
dfn2_avx_nonfused_probempi['NPY'] = (dfn2_avx_nonfused_probempi.GLB_Y//dfn2_avx_nonfused_probempi.LOC_Y)
dfn2_avx_nonfused_probempi['NPZ'] = (dfn2_avx_nonfused_probempi.GLB_Z//dfn2_avx_nonfused_probempi.LOC_Z)
dfn2_avx_nonfused_probempi['NPT'] = (dfn2_avx_nonfused_probempi.GLB_T//dfn2_avx_nonfused_probempi.LOC_T)
dfn2_avx_nonfused_probempi["Total_proc"] = dfn2_avx_nonfused_probempi["omp_thrd"]* dfn2_avx_nonfused_probempi.MPI_size
# dfn2_avx_nonfused_probempi.columns

### Data Summary

In [116]:
dfn2_avx_nonfused_probempi_summary = dfn2_avx_nonfused_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn2_avx_nonfused_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,16,16,2,2,2,2,256,120
1,32,8,4,2,2,2,256,69
2,64,4,8,2,2,2,256,35
3,128,2,16,2,2,2,256,16
4,248,1,31,2,2,2,248,4
5,256,1,32,2,2,2,256,7


### Cases Setup

In [123]:
dfn2AvxNonFusProb_16mpi_16omp = dfn2_avx_nonfused_probempi[(dfn2_avx_nonfused_probempi.omp_thrd == 16) & (dfn2_avx_nonfused_probempi.GLB_T/dfn2_avx_nonfused_probempi.LOC_T == 2) & (dfn2_avx_nonfused_probempi.GLB_X/dfn2_avx_nonfused_probempi.LOC_X == 2) & (dfn2_avx_nonfused_probempi.GLB_Y/dfn2_avx_nonfused_probempi.LOC_Y == 2)& (dfn2_avx_nonfused_probempi.GLB_Z/dfn2_avx_nonfused_probempi.LOC_Z == 2)]
dfn2AvxNonFusProb_32mpi_8omp = dfn2_avx_nonfused_probempi[(dfn2_avx_nonfused_probempi.omp_thrd == 8) & (dfn2_avx_nonfused_probempi.GLB_T/dfn2_avx_nonfused_probempi.LOC_T == 4) & (dfn2_avx_nonfused_probempi.GLB_X/dfn2_avx_nonfused_probempi.LOC_X == 2) & (dfn2_avx_nonfused_probempi.GLB_Y/dfn2_avx_nonfused_probempi.LOC_Y == 2)& (dfn2_avx_nonfused_probempi.GLB_Z/dfn2_avx_nonfused_probempi.LOC_Z == 2)]
dfn2AvxNonFusProb_64mpi_4omp = dfn2_avx_nonfused_probempi[(dfn2_avx_nonfused_probempi.omp_thrd == 4) & (dfn2_avx_nonfused_probempi.GLB_T/dfn2_avx_nonfused_probempi.LOC_T == 8) & (dfn2_avx_nonfused_probempi.GLB_X/dfn2_avx_nonfused_probempi.LOC_X == 2) & (dfn2_avx_nonfused_probempi.GLB_Y/dfn2_avx_nonfused_probempi.LOC_Y == 2)& (dfn2_avx_nonfused_probempi.GLB_Z/dfn2_avx_nonfused_probempi.LOC_Z == 2)]
dfn2AvxNonFusProb_128mpi_2omp = dfn2_avx_nonfused_probempi[(dfn2_avx_nonfused_probempi.omp_thrd == 2) & (dfn2_avx_nonfused_probempi.GLB_T/dfn2_avx_nonfused_probempi.LOC_T == 16) & (dfn2_avx_nonfused_probempi.GLB_X/dfn2_avx_nonfused_probempi.LOC_X == 2) & (dfn2_avx_nonfused_probempi.GLB_Y/dfn2_avx_nonfused_probempi.LOC_Y == 2)& (dfn2_avx_nonfused_probempi.GLB_Z/dfn2_avx_nonfused_probempi.LOC_Z == 2)]
dfn2AvxNonFusProb_256mpi_1omp = dfn2_avx_nonfused_probempi[(dfn2_avx_nonfused_probempi.omp_thrd == 1) & (dfn2_avx_nonfused_probempi.GLB_T/dfn2_avx_nonfused_probempi.LOC_T == 32) & (dfn2_avx_nonfused_probempi.GLB_X/dfn2_avx_nonfused_probempi.LOC_X == 2) & (dfn2_avx_nonfused_probempi.GLB_Y/dfn2_avx_nonfused_probempi.LOC_Y == 2)& (dfn2_avx_nonfused_probempi.GLB_Z/dfn2_avx_nonfused_probempi.LOC_Z == 2)]


## 4) NonFused_AVX_NonProbempi

### Adding Columns

In [124]:
dfn2_avx_nonfused_nonprobempi["N_Nodes"] = 2
dfn2_avx_nonfused_nonprobempi['NPX'] = (dfn2_avx_nonfused_nonprobempi.GLB_X//dfn2_avx_nonfused_nonprobempi.LOC_X)
dfn2_avx_nonfused_nonprobempi['NPY'] = (dfn2_avx_nonfused_nonprobempi.GLB_Y//dfn2_avx_nonfused_nonprobempi.LOC_Y)
dfn2_avx_nonfused_nonprobempi['NPZ'] = (dfn2_avx_nonfused_nonprobempi.GLB_Z//dfn2_avx_nonfused_nonprobempi.LOC_Z)
dfn2_avx_nonfused_nonprobempi['NPT'] = (dfn2_avx_nonfused_nonprobempi.GLB_T//dfn2_avx_nonfused_nonprobempi.LOC_T)
dfn2_avx_nonfused_nonprobempi["Total_proc"] = dfn2_avx_nonfused_nonprobempi["omp_thrd"]* dfn2_avx_nonfused_nonprobempi.MPI_size
# dfn2_avx_nonfused_nonprobempi.columns

### Data Summary

In [125]:
dfn2_avx_nonfused_nonprobempi_summary = dfn2_avx_nonfused_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn2_avx_nonfused_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,16,16,2,2,2,2,256,116
1,32,8,4,2,2,2,256,70
2,64,4,8,2,2,2,256,35
3,128,2,16,2,2,2,256,16
4,248,1,31,2,2,2,248,4
5,256,1,32,2,2,2,256,7


### Cases Setup

In [131]:
dfn2AvxNonFusNonProb_16mpi_16omp = dfn2_avx_nonfused_nonprobempi[(dfn2_avx_nonfused_nonprobempi.omp_thrd == 16) & (dfn2_avx_nonfused_nonprobempi.GLB_T/dfn2_avx_nonfused_nonprobempi.LOC_T == 2) & (dfn2_avx_nonfused_nonprobempi.GLB_X/dfn2_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn2_avx_nonfused_nonprobempi.GLB_Y/dfn2_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn2_avx_nonfused_nonprobempi.GLB_Z/dfn2_avx_nonfused_nonprobempi.LOC_Z == 2)]
dfn2AvxNonFusNonProb_32mpi_8omp = dfn2_avx_nonfused_nonprobempi[(dfn2_avx_nonfused_nonprobempi.omp_thrd == 8) & (dfn2_avx_nonfused_nonprobempi.GLB_T/dfn2_avx_nonfused_nonprobempi.LOC_T == 4) & (dfn2_avx_nonfused_nonprobempi.GLB_X/dfn2_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn2_avx_nonfused_nonprobempi.GLB_Y/dfn2_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn2_avx_nonfused_nonprobempi.GLB_Z/dfn2_avx_nonfused_nonprobempi.LOC_Z == 2)]
dfn2AvxNonFusNonProb_64mpi_4omp = dfn2_avx_nonfused_nonprobempi[(dfn2_avx_nonfused_nonprobempi.omp_thrd == 4) & (dfn2_avx_nonfused_nonprobempi.GLB_T/dfn2_avx_nonfused_nonprobempi.LOC_T == 8) & (dfn2_avx_nonfused_nonprobempi.GLB_X/dfn2_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn2_avx_nonfused_nonprobempi.GLB_Y/dfn2_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn2_avx_nonfused_nonprobempi.GLB_Z/dfn2_avx_nonfused_nonprobempi.LOC_Z == 2)]
dfn2AvxNonFusNonProb_128mpi_2omp = dfn2_avx_nonfused_nonprobempi[(dfn2_avx_nonfused_nonprobempi.omp_thrd == 2) & (dfn2_avx_nonfused_nonprobempi.GLB_T/dfn2_avx_nonfused_nonprobempi.LOC_T == 16) & (dfn2_avx_nonfused_nonprobempi.GLB_X/dfn2_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn2_avx_nonfused_nonprobempi.GLB_Y/dfn2_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn2_avx_nonfused_nonprobempi.GLB_Z/dfn2_avx_nonfused_nonprobempi.LOC_Z == 2)]
dfn2AvxNonFusNonProb_256mpi_1omp = dfn2_avx_nonfused_nonprobempi[(dfn2_avx_nonfused_nonprobempi.omp_thrd == 1) & (dfn2_avx_nonfused_nonprobempi.GLB_T/dfn2_avx_nonfused_nonprobempi.LOC_T == 32) & (dfn2_avx_nonfused_nonprobempi.GLB_X/dfn2_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn2_avx_nonfused_nonprobempi.GLB_Y/dfn2_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn2_avx_nonfused_nonprobempi.GLB_Z/dfn2_avx_nonfused_nonprobempi.LOC_Z == 2)]


## GFLOPS - NonFused-AVX: Probmpi vs Non-Probempi

In [132]:
fig = go.Figure() 

fig.add_trace(go.Scatter(x=(dfn2AvxNonFusNonProb_256mpi_1omp.Dirc_Data_Mov*(dfn2AvxNonFusNonProb_256mpi_1omp.MPI_size/2)), 
                         y=(dfn2AvxNonFusNonProb_256mpi_1omp.GFLOPS*(dfn2AvxNonFusNonProb_256mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_256mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


fig.add_trace(go.Scatter(x=(dfn2AvxNonFusProb_256mpi_1omp.Dirc_Data_Mov*(dfn2AvxNonFusProb_256mpi_1omp.MPI_size/2)), 
                         y=(dfn2AvxNonFusProb_256mpi_1omp.GFLOPS*(dfn2AvxNonFusProb_256mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_256mpi_1omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

# case 2
fig.add_trace(go.Scatter(x=(dfn2AvxNonFusNonProb_128mpi_2omp.Dirc_Data_Mov*(dfn2AvxNonFusNonProb_128mpi_2omp.MPI_size/2)), 
                         y=(dfn2AvxNonFusNonProb_128mpi_2omp.GFLOPS*(dfn2AvxNonFusNonProb_128mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_128mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn2AvxNonFusProb_128mpi_2omp.Dirc_Data_Mov*(dfn2AvxNonFusProb_128mpi_2omp.MPI_size/2)), 
                         y=(dfn2AvxNonFusProb_128mpi_2omp.GFLOPS*(dfn2AvxNonFusProb_128mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_128mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

# case 3
fig.add_trace(go.Scatter(x=(dfn2AvxNonFusNonProb_64mpi_4omp.Dirc_Data_Mov*(dfn2AvxNonFusNonProb_64mpi_4omp.MPI_size/2)), 
                         y=(dfn2AvxNonFusNonProb_64mpi_4omp.GFLOPS*(dfn2AvxNonFusNonProb_64mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_64mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))

fig.add_trace(go.Scatter(x=(dfn2AvxNonFusProb_64mpi_4omp.Dirc_Data_Mov*(dfn2AvxNonFusProb_64mpi_4omp.MPI_size/2)), 
                         y=(dfn2AvxNonFusProb_64mpi_4omp.GFLOPS*(dfn2AvxNonFusProb_64mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_64mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

# case 4
fig.add_trace(go.Scatter(x=(dfn2AvxNonFusNonProb_32mpi_8omp.Dirc_Data_Mov*(dfn2AvxNonFusNonProb_32mpi_8omp.MPI_size/2)), 
                         y=(dfn2AvxNonFusNonProb_32mpi_8omp.GFLOPS*(dfn2AvxNonFusNonProb_32mpi_8omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_32mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='red',width=2))))

fig.add_trace(go.Scatter(x=(dfn2AvxNonFusProb_32mpi_8omp.Dirc_Data_Mov*(dfn2AvxNonFusProb_32mpi_8omp.MPI_size/2)), 
                         y=(dfn2AvxNonFusProb_32mpi_8omp.GFLOPS*(dfn2AvxNonFusProb_32mpi_8omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_32mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='green',width=2))))

# case 5

fig.add_trace(go.Scatter(x=(dfn2AvxNonFusNonProb_16mpi_16omp.Dirc_Data_Mov*(dfn2AvxNonFusNonProb_16mpi_16omp.MPI_size/2)), 
                         y=(dfn2AvxNonFusNonProb_16mpi_16omp.GFLOPS*(dfn2AvxNonFusNonProb_16mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_16mpi_16omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='chocolate',width=2))))

fig.add_trace(go.Scatter(x=(dfn2AvxNonFusProb_16mpi_16omp.Dirc_Data_Mov*(dfn2AvxNonFusProb_16mpi_16omp.MPI_size/2)), 
                         y=(dfn2AvxNonFusProb_16mpi_16omp.GFLOPS*(dfn2AvxNonFusProb_16mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_16mpi_16omp',
                         line=dict(color='darkviolet',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='crimson',width=2))))



# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"NonFused-AVX (2-Nodes): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1))

## 5) Newgeo_AVX_Probempi

### Adding Columns

In [133]:
dfn2_avx_newgeo_probempi["N_Nodes"] = 2
dfn2_avx_newgeo_probempi['NPX'] = (dfn2_avx_newgeo_probempi.GLB_X//dfn2_avx_newgeo_probempi.LOC_X)
dfn2_avx_newgeo_probempi['NPY'] = (dfn2_avx_newgeo_probempi.GLB_Y//dfn2_avx_newgeo_probempi.LOC_Y)
dfn2_avx_newgeo_probempi['NPZ'] = (dfn2_avx_newgeo_probempi.GLB_Z//dfn2_avx_newgeo_probempi.LOC_Z)
dfn2_avx_newgeo_probempi['NPT'] = (dfn2_avx_newgeo_probempi.GLB_T//dfn2_avx_newgeo_probempi.LOC_T)
dfn2_avx_newgeo_probempi["Total_proc"] = dfn2_avx_newgeo_probempi["omp_thrd"]* dfn2_avx_newgeo_probempi.MPI_size
# dfn2_avx_newgeo_probempi.columns

### Data Summary

In [134]:
dfn2_avx_newgeo_probempi_summary = dfn2_avx_newgeo_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn2_avx_newgeo_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,16,16,2,2,2,2,256,139
1,32,8,4,2,2,2,256,68
2,64,4,8,2,2,2,256,34
3,128,2,16,2,2,2,256,17
4,248,1,31,2,2,2,248,4
5,256,1,32,2,2,2,256,7


### Cases Setup

In [135]:
dfn2AvxNewgeoProb_16mpi_16omp = dfn2_avx_newgeo_probempi[(dfn2_avx_newgeo_probempi.omp_thrd == 16) & (dfn2_avx_newgeo_probempi.GLB_T/dfn2_avx_newgeo_probempi.LOC_T == 2) & (dfn2_avx_newgeo_probempi.GLB_X/dfn2_avx_newgeo_probempi.LOC_X == 2) & (dfn2_avx_newgeo_probempi.GLB_Y/dfn2_avx_newgeo_probempi.LOC_Y == 2)& (dfn2_avx_newgeo_probempi.GLB_Z/dfn2_avx_newgeo_probempi.LOC_Z == 2)]
dfn2AvxNewgeoProb_32mpi_8omp = dfn2_avx_newgeo_probempi[(dfn2_avx_newgeo_probempi.omp_thrd == 8) & (dfn2_avx_newgeo_probempi.GLB_T/dfn2_avx_newgeo_probempi.LOC_T == 4) & (dfn2_avx_newgeo_probempi.GLB_X/dfn2_avx_newgeo_probempi.LOC_X == 2) & (dfn2_avx_newgeo_probempi.GLB_Y/dfn2_avx_newgeo_probempi.LOC_Y == 2)& (dfn2_avx_newgeo_probempi.GLB_Z/dfn2_avx_newgeo_probempi.LOC_Z == 2)]
dfn2AvxNewgeoProb_64mpi_4omp = dfn2_avx_newgeo_probempi[(dfn2_avx_newgeo_probempi.omp_thrd == 4) & (dfn2_avx_newgeo_probempi.GLB_T/dfn2_avx_newgeo_probempi.LOC_T == 8) & (dfn2_avx_newgeo_probempi.GLB_X/dfn2_avx_newgeo_probempi.LOC_X == 2) & (dfn2_avx_newgeo_probempi.GLB_Y/dfn2_avx_newgeo_probempi.LOC_Y == 2)& (dfn2_avx_newgeo_probempi.GLB_Z/dfn2_avx_newgeo_probempi.LOC_Z == 2)]
dfn2AvxNewgeoProb_128mpi_2omp = dfn2_avx_newgeo_probempi[(dfn2_avx_newgeo_probempi.omp_thrd == 2) & (dfn2_avx_newgeo_probempi.GLB_T/dfn2_avx_newgeo_probempi.LOC_T == 16) & (dfn2_avx_newgeo_probempi.GLB_X/dfn2_avx_newgeo_probempi.LOC_X == 2) & (dfn2_avx_newgeo_probempi.GLB_Y/dfn2_avx_newgeo_probempi.LOC_Y == 2)& (dfn2_avx_newgeo_probempi.GLB_Z/dfn2_avx_newgeo_probempi.LOC_Z == 2)]
dfn2AvxNewgeoProb_256mpi_1omp = dfn2_avx_newgeo_probempi[(dfn2_avx_newgeo_probempi.omp_thrd == 1) & (dfn2_avx_newgeo_probempi.GLB_T/dfn2_avx_newgeo_probempi.LOC_T == 32) & (dfn2_avx_newgeo_probempi.GLB_X/dfn2_avx_newgeo_probempi.LOC_X == 2) & (dfn2_avx_newgeo_probempi.GLB_Y/dfn2_avx_newgeo_probempi.LOC_Y == 2)& (dfn2_avx_newgeo_probempi.GLB_Z/dfn2_avx_newgeo_probempi.LOC_Z == 2)]


## 6) Newgeo_AVX_NonProbempi

### Adding Columns

In [136]:
dfn2_avx_newgeo_nonprobempi["N_Nodes"] = 2
dfn2_avx_newgeo_nonprobempi['NPX'] = (dfn2_avx_newgeo_nonprobempi.GLB_X//dfn2_avx_newgeo_nonprobempi.LOC_X)
dfn2_avx_newgeo_nonprobempi['NPY'] = (dfn2_avx_newgeo_nonprobempi.GLB_Y//dfn2_avx_newgeo_nonprobempi.LOC_Y)
dfn2_avx_newgeo_nonprobempi['NPZ'] = (dfn2_avx_newgeo_nonprobempi.GLB_Z//dfn2_avx_newgeo_nonprobempi.LOC_Z)
dfn2_avx_newgeo_nonprobempi['NPT'] = (dfn2_avx_newgeo_nonprobempi.GLB_T//dfn2_avx_newgeo_nonprobempi.LOC_T)
dfn2_avx_newgeo_nonprobempi["Total_proc"] = dfn2_avx_newgeo_nonprobempi["omp_thrd"]* dfn2_avx_newgeo_nonprobempi.MPI_size
# dfn2_avx_newgeo_nonprobempi.columns

### Data Summary

In [137]:
dfn2_avx_newgeo_nonprobempi_summary = dfn2_avx_newgeo_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn2_avx_newgeo_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,16,16,2,2,2,2,256,139
1,32,8,4,2,2,2,256,70
2,64,4,8,2,2,2,256,35
3,128,2,16,2,2,2,256,17
4,248,1,31,2,2,2,248,4
5,256,1,32,2,2,2,256,7


### Cases Setup

In [143]:
dfn2AvxNewgeoNonProb_16mpi_16omp = dfn2_avx_newgeo_nonprobempi[(dfn2_avx_newgeo_nonprobempi.omp_thrd == 16) & (dfn2_avx_newgeo_nonprobempi.GLB_T/dfn2_avx_newgeo_nonprobempi.LOC_T == 2) & (dfn2_avx_newgeo_nonprobempi.GLB_X/dfn2_avx_newgeo_nonprobempi.LOC_X == 2) & (dfn2_avx_newgeo_nonprobempi.GLB_Y/dfn2_avx_newgeo_nonprobempi.LOC_Y == 2)& (dfn2_avx_newgeo_nonprobempi.GLB_Z/dfn2_avx_newgeo_nonprobempi.LOC_Z == 2)]
dfn2AvxNewgeoNonProb_32mpi_8omp = dfn2_avx_newgeo_nonprobempi[(dfn2_avx_newgeo_nonprobempi.omp_thrd == 8) & (dfn2_avx_newgeo_nonprobempi.GLB_T/dfn2_avx_newgeo_nonprobempi.LOC_T == 4) & (dfn2_avx_newgeo_nonprobempi.GLB_X/dfn2_avx_newgeo_nonprobempi.LOC_X == 2) & (dfn2_avx_newgeo_nonprobempi.GLB_Y/dfn2_avx_newgeo_nonprobempi.LOC_Y == 2)& (dfn2_avx_newgeo_nonprobempi.GLB_Z/dfn2_avx_newgeo_nonprobempi.LOC_Z == 2)]
dfn2AvxNewgeoNonProb_64mpi_4omp = dfn2_avx_newgeo_nonprobempi[(dfn2_avx_newgeo_nonprobempi.omp_thrd == 4) & (dfn2_avx_newgeo_nonprobempi.GLB_T/dfn2_avx_newgeo_nonprobempi.LOC_T == 8) & (dfn2_avx_newgeo_nonprobempi.GLB_X/dfn2_avx_newgeo_nonprobempi.LOC_X == 2) & (dfn2_avx_newgeo_nonprobempi.GLB_Y/dfn2_avx_newgeo_nonprobempi.LOC_Y == 2)& (dfn2_avx_newgeo_nonprobempi.GLB_Z/dfn2_avx_newgeo_nonprobempi.LOC_Z == 2)]
dfn2AvxNewgeoNonProb_128mpi_2omp = dfn2_avx_newgeo_nonprobempi[(dfn2_avx_newgeo_nonprobempi.omp_thrd == 2) & (dfn2_avx_newgeo_nonprobempi.GLB_T/dfn2_avx_newgeo_nonprobempi.LOC_T == 16) & (dfn2_avx_newgeo_nonprobempi.GLB_X/dfn2_avx_newgeo_nonprobempi.LOC_X == 2) & (dfn2_avx_newgeo_nonprobempi.GLB_Y/dfn2_avx_newgeo_nonprobempi.LOC_Y == 2)& (dfn2_avx_newgeo_nonprobempi.GLB_Z/dfn2_avx_newgeo_nonprobempi.LOC_Z == 2)]
dfn2AvxNewgeoNonProb_256mpi_1omp = dfn2_avx_newgeo_nonprobempi[(dfn2_avx_newgeo_nonprobempi.omp_thrd == 1) & (dfn2_avx_newgeo_nonprobempi.GLB_T/dfn2_avx_newgeo_nonprobempi.LOC_T == 32) & (dfn2_avx_newgeo_nonprobempi.GLB_X/dfn2_avx_newgeo_nonprobempi.LOC_X == 2) & (dfn2_avx_newgeo_nonprobempi.GLB_Y/dfn2_avx_newgeo_nonprobempi.LOC_Y == 2)& (dfn2_avx_newgeo_nonprobempi.GLB_Z/dfn2_avx_newgeo_nonprobempi.LOC_Z == 2)]


## GFLOPS - Newgeo-AVX: Probmpi vs Non-Probempi

In [144]:
fig = go.Figure() 

fig.add_trace(go.Scatter(x=(dfn2AvxNewgeoNonProb_256mpi_1omp.Dirc_Data_Mov*(dfn2AvxNewgeoNonProb_256mpi_1omp.MPI_size/2)), 
                         y=(dfn2AvxNewgeoNonProb_256mpi_1omp.GFLOPS*(dfn2AvxNewgeoNonProb_256mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxNewgeoNonProb_256mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


fig.add_trace(go.Scatter(x=(dfn2AvxNewgeoProb_256mpi_1omp.Dirc_Data_Mov*(dfn2AvxNewgeoProb_256mpi_1omp.MPI_size/2)), 
                         y=(dfn2AvxNewgeoProb_256mpi_1omp.GFLOPS*(dfn2AvxNewgeoProb_256mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_256mpi_1omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

# case 2
fig.add_trace(go.Scatter(x=(dfn2AvxNewgeoNonProb_128mpi_2omp.Dirc_Data_Mov*(dfn2AvxNewgeoNonProb_128mpi_2omp.MPI_size/2)), 
                         y=(dfn2AvxNewgeoNonProb_128mpi_2omp.GFLOPS*(dfn2AvxNewgeoNonProb_128mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxNewgeoNonProb_128mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn2AvxNewgeoProb_128mpi_2omp.Dirc_Data_Mov*(dfn2AvxNewgeoProb_128mpi_2omp.MPI_size/2)), 
                         y=(dfn2AvxNewgeoProb_128mpi_2omp.GFLOPS*(dfn2AvxNewgeoProb_128mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxNewgeoProb_128mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

# case 3
fig.add_trace(go.Scatter(x=(dfn2AvxNewgeoNonProb_64mpi_4omp.Dirc_Data_Mov*(dfn2AvxNewgeoNonProb_64mpi_4omp.MPI_size/2)), 
                         y=(dfn2AvxNewgeoNonProb_64mpi_4omp.GFLOPS*(dfn2AvxNewgeoNonProb_64mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxNewgeoNonProb_64mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))

fig.add_trace(go.Scatter(x=(dfn2AvxNewgeoProb_64mpi_4omp.Dirc_Data_Mov*(dfn2AvxNewgeoProb_64mpi_4omp.MPI_size/2)), 
                         y=(dfn2AvxNewgeoProb_64mpi_4omp.GFLOPS*(dfn2AvxNewgeoProb_64mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxNewgeoProb_64mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

# case 4
fig.add_trace(go.Scatter(x=(dfn2AvxNewgeoNonProb_32mpi_8omp.Dirc_Data_Mov*(dfn2AvxNewgeoNonProb_32mpi_8omp.MPI_size/2)), 
                         y=(dfn2AvxNewgeoNonProb_32mpi_8omp.GFLOPS*(dfn2AvxNewgeoNonProb_32mpi_8omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_32mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='red',width=2))))

fig.add_trace(go.Scatter(x=(dfn2AvxNewgeoProb_32mpi_8omp.Dirc_Data_Mov*(dfn2AvxNewgeoProb_32mpi_8omp.MPI_size/2)), 
                         y=(dfn2AvxNewgeoProb_32mpi_8omp.GFLOPS*(dfn2AvxNewgeoProb_32mpi_8omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxNewgeoProb_32mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='green',width=2))))

# case 5

fig.add_trace(go.Scatter(x=(dfn2AvxNewgeoNonProb_16mpi_16omp.Dirc_Data_Mov*(dfn2AvxNewgeoNonProb_16mpi_16omp.MPI_size/2)), 
                         y=(dfn2AvxNewgeoNonProb_16mpi_16omp.GFLOPS*(dfn2AvxNewgeoNonProb_16mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxNewgeoNonProb_16mpi_16omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='chocolate',width=2))))

fig.add_trace(go.Scatter(x=(dfn2AvxNewgeoProb_16mpi_16omp.Dirc_Data_Mov*(dfn2AvxNewgeoProb_16mpi_16omp.MPI_size/2)), 
                         y=(dfn2AvxNewgeoProb_16mpi_16omp.GFLOPS*(dfn2AvxNewgeoProb_16mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='AvxNewgeoProb_16mpi_16omp',
                         line=dict(color='darkviolet',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='crimson',width=2))))



# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"Newgeo-AVX (2-Nodes): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1))

# HIPPO VECT CASES

## 1) Fused_VECT_Probmpi

### Adding Columns

In [170]:
dfn2_vect_fused_probmpi["N_Nodes"] = 2
dfn2_vect_fused_probmpi['NPX'] = (dfn2_vect_fused_probmpi.GLB_X//dfn2_vect_fused_probmpi.LOC_X)
dfn2_vect_fused_probmpi['NPY'] = (dfn2_vect_fused_probmpi.GLB_Y//dfn2_vect_fused_probmpi.LOC_Y)
dfn2_vect_fused_probmpi['NPZ'] = (dfn2_vect_fused_probmpi.GLB_Z//dfn2_vect_fused_probmpi.LOC_Z)
dfn2_vect_fused_probmpi['NPT'] = (dfn2_vect_fused_probmpi.GLB_T//dfn2_vect_fused_probmpi.LOC_T)
dfn2_vect_fused_probmpi["Total_proc"] = dfn2_vect_fused_probmpi["omp_thrd"]* dfn2_vect_fused_probmpi.MPI_size
# dfn2_vect_fused_probmpi.columns

### Data Summary

In [154]:
dfn2_vect_fused_probmpi_summary = dfn2_vect_fused_probmpi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn2_vect_fused_probmpi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,16,16,2,2,2,2,256,140
1,32,8,4,2,2,2,256,70
2,64,4,8,2,2,2,256,35
3,128,2,16,2,2,2,256,17
4,248,1,31,2,2,2,248,4
5,256,1,32,2,2,2,256,7


### Cases Setup

In [160]:
dfn2VectFusProb_16mpi_16omp = dfn2_vect_fused_probmpi[(dfn2_vect_fused_probmpi.omp_thrd == 16) & (dfn2_vect_fused_probmpi.GLB_T/dfn2_vect_fused_probmpi.LOC_T == 2) & (dfn2_vect_fused_probmpi.GLB_X/dfn2_vect_fused_probmpi.LOC_X == 2) & (dfn2_vect_fused_probmpi.GLB_Y/dfn2_vect_fused_probmpi.LOC_Y == 2)& (dfn2_vect_fused_probmpi.GLB_Z/dfn2_vect_fused_probmpi.LOC_Z == 2)]
dfn2VectFusProb_32mpi_8omp = dfn2_vect_fused_probmpi[(dfn2_vect_fused_probmpi.omp_thrd == 8) & (dfn2_vect_fused_probmpi.GLB_T/dfn2_vect_fused_probmpi.LOC_T == 4) & (dfn2_vect_fused_probmpi.GLB_X/dfn2_vect_fused_probmpi.LOC_X == 2) & (dfn2_vect_fused_probmpi.GLB_Y/dfn2_vect_fused_probmpi.LOC_Y == 2)& (dfn2_vect_fused_probmpi.GLB_Z/dfn2_vect_fused_probmpi.LOC_Z == 2)]
dfn2VectFusProb_64mpi_4omp = dfn2_vect_fused_probmpi[(dfn2_vect_fused_probmpi.omp_thrd == 4) & (dfn2_vect_fused_probmpi.GLB_T/dfn2_vect_fused_probmpi.LOC_T == 8) & (dfn2_vect_fused_probmpi.GLB_X/dfn2_vect_fused_probmpi.LOC_X == 2) & (dfn2_vect_fused_probmpi.GLB_Y/dfn2_vect_fused_probmpi.LOC_Y == 2)& (dfn2_vect_fused_probmpi.GLB_Z/dfn2_vect_fused_probmpi.LOC_Z == 2)]
dfn2VectFusProb_128mpi_2omp = dfn2_vect_fused_probmpi[(dfn2_vect_fused_probmpi.omp_thrd == 2) & (dfn2_vect_fused_probmpi.GLB_T/dfn2_vect_fused_probmpi.LOC_T == 16) & (dfn2_vect_fused_probmpi.GLB_X/dfn2_vect_fused_probmpi.LOC_X == 2) & (dfn2_vect_fused_probmpi.GLB_Y/dfn2_vect_fused_probmpi.LOC_Y == 2)& (dfn2_vect_fused_probmpi.GLB_Z/dfn2_vect_fused_probmpi.LOC_Z == 2)]
dfn2VectFusProb_256mpi_1omp = dfn2_vect_fused_probmpi[(dfn2_vect_fused_probmpi.omp_thrd == 1) & (dfn2_vect_fused_probmpi.GLB_T/dfn2_vect_fused_probmpi.LOC_T == 32) & (dfn2_vect_fused_probmpi.GLB_X/dfn2_vect_fused_probmpi.LOC_X == 2) & (dfn2_vect_fused_probmpi.GLB_Y/dfn2_vect_fused_probmpi.LOC_Y == 2)& (dfn2_vect_fused_probmpi.GLB_Z/dfn2_vect_fused_probmpi.LOC_Z == 2)]


## 2) Fused_VECT_NonProbmpi

### Adding Columns

In [162]:
dfn2_vect_fused_nonprobmpi["N_Nodes"] = 2
dfn2_vect_fused_nonprobmpi['NPX'] = (dfn2_vect_fused_nonprobmpi.GLB_X//dfn2_vect_fused_nonprobmpi.LOC_X)
dfn2_vect_fused_nonprobmpi['NPY'] = (dfn2_vect_fused_nonprobmpi.GLB_Y//dfn2_vect_fused_nonprobmpi.LOC_Y)
dfn2_vect_fused_nonprobmpi['NPZ'] = (dfn2_vect_fused_nonprobmpi.GLB_Z//dfn2_vect_fused_nonprobmpi.LOC_Z)
dfn2_vect_fused_nonprobmpi['NPT'] = (dfn2_vect_fused_nonprobmpi.GLB_T//dfn2_vect_fused_nonprobmpi.LOC_T)
dfn2_vect_fused_nonprobmpi["Total_proc"] = dfn2_vect_fused_nonprobmpi["omp_thrd"]* dfn2_vect_fused_nonprobmpi.MPI_size
# dfn2_vect_fused_nonprobmpi.columns

### Data Summary

In [163]:
dfn2_vect_fused_nonprobmpi_summary = dfn2_vect_fused_nonprobmpi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn2_vect_fused_nonprobmpi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,16,16,2,2,2,2,256,139
1,32,8,4,2,2,2,256,70
2,64,4,8,2,2,2,256,35
3,128,2,16,2,2,2,256,16
4,248,1,31,2,2,2,248,4
5,256,1,32,2,2,2,256,6


### Cases Setup

In [169]:
dfn2VectFusNonProb_16mpi_16omp = dfn2_vect_fused_nonprobmpi[(dfn2_vect_fused_nonprobmpi.omp_thrd == 16) & (dfn2_vect_fused_nonprobmpi.GLB_T/dfn2_vect_fused_nonprobmpi.LOC_T == 2) & (dfn2_vect_fused_nonprobmpi.GLB_X/dfn2_vect_fused_nonprobmpi.LOC_X == 2) & (dfn2_vect_fused_nonprobmpi.GLB_Y/dfn2_vect_fused_nonprobmpi.LOC_Y == 2)& (dfn2_vect_fused_nonprobmpi.GLB_Z/dfn2_vect_fused_nonprobmpi.LOC_Z == 2)]
dfn2VectFusNonProb_32mpi_8omp = dfn2_vect_fused_nonprobmpi[(dfn2_vect_fused_nonprobmpi.omp_thrd == 8) & (dfn2_vect_fused_nonprobmpi.GLB_T/dfn2_vect_fused_nonprobmpi.LOC_T == 4) & (dfn2_vect_fused_nonprobmpi.GLB_X/dfn2_vect_fused_nonprobmpi.LOC_X == 2) & (dfn2_vect_fused_nonprobmpi.GLB_Y/dfn2_vect_fused_nonprobmpi.LOC_Y == 2)& (dfn2_vect_fused_nonprobmpi.GLB_Z/dfn2_vect_fused_nonprobmpi.LOC_Z == 2)]
dfn2VectFusNonProb_64mpi_4omp = dfn2_vect_fused_nonprobmpi[(dfn2_vect_fused_nonprobmpi.omp_thrd == 4) & (dfn2_vect_fused_nonprobmpi.GLB_T/dfn2_vect_fused_nonprobmpi.LOC_T == 8) & (dfn2_vect_fused_nonprobmpi.GLB_X/dfn2_vect_fused_nonprobmpi.LOC_X == 2) & (dfn2_vect_fused_nonprobmpi.GLB_Y/dfn2_vect_fused_nonprobmpi.LOC_Y == 2)& (dfn2_vect_fused_nonprobmpi.GLB_Z/dfn2_vect_fused_nonprobmpi.LOC_Z == 2)]
dfn2VectFusNonProb_128mpi_2omp = dfn2_vect_fused_nonprobmpi[(dfn2_vect_fused_nonprobmpi.omp_thrd == 2) & (dfn2_vect_fused_nonprobmpi.GLB_T/dfn2_vect_fused_nonprobmpi.LOC_T == 16) & (dfn2_vect_fused_nonprobmpi.GLB_X/dfn2_vect_fused_nonprobmpi.LOC_X == 2) & (dfn2_vect_fused_nonprobmpi.GLB_Y/dfn2_vect_fused_nonprobmpi.LOC_Y == 2)& (dfn2_vect_fused_nonprobmpi.GLB_Z/dfn2_vect_fused_nonprobmpi.LOC_Z == 2)]
dfn2VectFusNonProb_256mpi_1omp = dfn2_vect_fused_nonprobmpi[(dfn2_vect_fused_nonprobmpi.omp_thrd == 1) & (dfn2_vect_fused_nonprobmpi.GLB_T/dfn2_vect_fused_nonprobmpi.LOC_T == 32) & (dfn2_vect_fused_nonprobmpi.GLB_X/dfn2_vect_fused_nonprobmpi.LOC_X == 2) & (dfn2_vect_fused_nonprobmpi.GLB_Y/dfn2_vect_fused_nonprobmpi.LOC_Y == 2)& (dfn2_vect_fused_nonprobmpi.GLB_Z/dfn2_vect_fused_nonprobmpi.LOC_Z == 2)]


## GFLOPS - Fused VECT: probempi vs nonprobempi

In [171]:
fig = go.Figure() 

fig.add_trace(go.Scatter(x=(dfn2VectFusNonProb_256mpi_1omp.Dirc_Data_Mov*(dfn2VectFusNonProb_256mpi_1omp.MPI_size/2)), 
                         y=(dfn2VectFusNonProb_256mpi_1omp.GFLOPS*(dfn2VectFusNonProb_256mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_256mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


fig.add_trace(go.Scatter(x=(dfn2VectFusProb_256mpi_1omp.Dirc_Data_Mov*(dfn2VectFusProb_256mpi_1omp.MPI_size/2)), 
                         y=(dfn2VectFusProb_256mpi_1omp.GFLOPS*(dfn2VectFusProb_256mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectFusProb_256mpi_1omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

# case 2
fig.add_trace(go.Scatter(x=(dfn2VectFusNonProb_128mpi_2omp.Dirc_Data_Mov*(dfn2VectFusNonProb_128mpi_2omp.MPI_size/2)), 
                         y=(dfn2VectFusNonProb_128mpi_2omp.GFLOPS*(dfn2VectFusNonProb_128mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_128mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn2VectFusProb_128mpi_2omp.Dirc_Data_Mov*(dfn2VectFusProb_128mpi_2omp.MPI_size/2)), 
                         y=(dfn2VectFusProb_128mpi_2omp.GFLOPS*(dfn2VectFusProb_128mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectFusProb_128mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

# case 3
fig.add_trace(go.Scatter(x=(dfn2VectFusNonProb_64mpi_4omp.Dirc_Data_Mov*(dfn2VectFusNonProb_64mpi_4omp.MPI_size/2)), 
                         y=(dfn2VectFusNonProb_64mpi_4omp.GFLOPS*(dfn2VectFusNonProb_64mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_64mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))

fig.add_trace(go.Scatter(x=(dfn2VectFusProb_64mpi_4omp.Dirc_Data_Mov*(dfn2VectFusProb_64mpi_4omp.MPI_size/2)), 
                         y=(dfn2VectFusProb_64mpi_4omp.GFLOPS*(dfn2VectFusProb_64mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectFusProb_64mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

# case 4
fig.add_trace(go.Scatter(x=(dfn2VectFusNonProb_32mpi_8omp.Dirc_Data_Mov*(dfn2VectFusNonProb_32mpi_8omp.MPI_size/2)), 
                         y=(dfn2VectFusNonProb_32mpi_8omp.GFLOPS*(dfn2VectFusNonProb_32mpi_8omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_32mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='red',width=2))))

fig.add_trace(go.Scatter(x=(dfn2VectFusProb_32mpi_8omp.Dirc_Data_Mov*(dfn2VectFusProb_32mpi_8omp.MPI_size/2)), 
                         y=(dfn2VectFusProb_32mpi_8omp.GFLOPS*(dfn2VectFusProb_32mpi_8omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectFusProb_32mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='green',width=2))))

# case 5

fig.add_trace(go.Scatter(x=(dfn2VectFusNonProb_16mpi_16omp.Dirc_Data_Mov*(dfn2VectFusNonProb_16mpi_16omp.MPI_size/2)), 
                         y=(dfn2VectFusNonProb_16mpi_16omp.GFLOPS*(dfn2VectFusNonProb_16mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_16mpi_16omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='chocolate',width=2))))

fig.add_trace(go.Scatter(x=(dfn2VectFusProb_16mpi_16omp.Dirc_Data_Mov*(dfn2VectFusProb_16mpi_16omp.MPI_size/2)), 
                         y=(dfn2VectFusProb_16mpi_16omp.GFLOPS*(dfn2VectFusProb_16mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectFusProb_16mpi_16omp',
                         line=dict(color='darkviolet',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='crimson',width=2))))



# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"Fused VECT (2-Nodes): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1))

## 3) NonFused_VECT_Probmpi

### Adding Columns

In [172]:
dfn2_vect_nonfused_probempi["N_Nodes"] = 2
dfn2_vect_nonfused_probempi['NPX'] = (dfn2_vect_nonfused_probempi.GLB_X//dfn2_vect_nonfused_probempi.LOC_X)
dfn2_vect_nonfused_probempi['NPY'] = (dfn2_vect_nonfused_probempi.GLB_Y//dfn2_vect_nonfused_probempi.LOC_Y)
dfn2_vect_nonfused_probempi['NPZ'] = (dfn2_vect_nonfused_probempi.GLB_Z//dfn2_vect_nonfused_probempi.LOC_Z)
dfn2_vect_nonfused_probempi['NPT'] = (dfn2_vect_nonfused_probempi.GLB_T//dfn2_vect_nonfused_probempi.LOC_T)
dfn2_vect_nonfused_probempi["Total_proc"] = dfn2_vect_nonfused_probempi["omp_thrd"]* dfn2_vect_nonfused_probempi.MPI_size
# dfn2_vect_nonfused_probempi.columns

### Data Summary

In [173]:
dfn2_vect_nonfused_probempi_summary = dfn2_vect_nonfused_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn2_vect_nonfused_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,16,16,2,2,2,2,256,113
1,32,8,4,2,2,2,256,69
2,64,4,8,2,2,2,256,35
3,128,2,16,2,2,2,256,16
4,248,1,31,2,2,2,248,4
5,256,1,32,2,2,2,256,7


### Cases Setup

In [174]:
dfn2VectNonFusProb_16mpi_16omp = dfn2_vect_nonfused_probempi[(dfn2_vect_nonfused_probempi.omp_thrd == 16) & (dfn2_vect_nonfused_probempi.GLB_T/dfn2_vect_nonfused_probempi.LOC_T == 2) & (dfn2_vect_nonfused_probempi.GLB_X/dfn2_vect_nonfused_probempi.LOC_X == 2) & (dfn2_vect_nonfused_probempi.GLB_Y/dfn2_vect_nonfused_probempi.LOC_Y == 2)& (dfn2_vect_nonfused_probempi.GLB_Z/dfn2_vect_nonfused_probempi.LOC_Z == 2)]
dfn2VectNonFusProb_32mpi_8omp = dfn2_vect_nonfused_probempi[(dfn2_vect_nonfused_probempi.omp_thrd == 8) & (dfn2_vect_nonfused_probempi.GLB_T/dfn2_vect_nonfused_probempi.LOC_T == 4) & (dfn2_vect_nonfused_probempi.GLB_X/dfn2_vect_nonfused_probempi.LOC_X == 2) & (dfn2_vect_nonfused_probempi.GLB_Y/dfn2_vect_nonfused_probempi.LOC_Y == 2)& (dfn2_vect_nonfused_probempi.GLB_Z/dfn2_vect_nonfused_probempi.LOC_Z == 2)]
dfn2VectNonFusProb_64mpi_4omp = dfn2_vect_nonfused_probempi[(dfn2_vect_nonfused_probempi.omp_thrd == 4) & (dfn2_vect_nonfused_probempi.GLB_T/dfn2_vect_nonfused_probempi.LOC_T == 8) & (dfn2_vect_nonfused_probempi.GLB_X/dfn2_vect_nonfused_probempi.LOC_X == 2) & (dfn2_vect_nonfused_probempi.GLB_Y/dfn2_vect_nonfused_probempi.LOC_Y == 2)& (dfn2_vect_nonfused_probempi.GLB_Z/dfn2_vect_nonfused_probempi.LOC_Z == 2)]
dfn2VectNonFusProb_128mpi_2omp = dfn2_vect_nonfused_probempi[(dfn2_vect_nonfused_probempi.omp_thrd == 2) & (dfn2_vect_nonfused_probempi.GLB_T/dfn2_vect_nonfused_probempi.LOC_T == 16) & (dfn2_vect_nonfused_probempi.GLB_X/dfn2_vect_nonfused_probempi.LOC_X == 2) & (dfn2_vect_nonfused_probempi.GLB_Y/dfn2_vect_nonfused_probempi.LOC_Y == 2)& (dfn2_vect_nonfused_probempi.GLB_Z/dfn2_vect_nonfused_probempi.LOC_Z == 2)]
dfn2VectNonFusProb_256mpi_1omp = dfn2_vect_nonfused_probempi[(dfn2_vect_nonfused_probempi.omp_thrd == 1) & (dfn2_vect_nonfused_probempi.GLB_T/dfn2_vect_nonfused_probempi.LOC_T == 32) & (dfn2_vect_nonfused_probempi.GLB_X/dfn2_vect_nonfused_probempi.LOC_X == 2) & (dfn2_vect_nonfused_probempi.GLB_Y/dfn2_vect_nonfused_probempi.LOC_Y == 2)& (dfn2_vect_nonfused_probempi.GLB_Z/dfn2_vect_nonfused_probempi.LOC_Z == 2)]


## 4) NonFused_VECT_NonProbmpi

### Adding Columns

In [175]:
dfn2_vect_nonfused_nonprobempi["N_Nodes"] = 2
dfn2_vect_nonfused_nonprobempi['NPX'] = (dfn2_vect_nonfused_nonprobempi.GLB_X//dfn2_vect_nonfused_nonprobempi.LOC_X)
dfn2_vect_nonfused_nonprobempi['NPY'] = (dfn2_vect_nonfused_nonprobempi.GLB_Y//dfn2_vect_nonfused_nonprobempi.LOC_Y)
dfn2_vect_nonfused_nonprobempi['NPZ'] = (dfn2_vect_nonfused_nonprobempi.GLB_Z//dfn2_vect_nonfused_nonprobempi.LOC_Z)
dfn2_vect_nonfused_nonprobempi['NPT'] = (dfn2_vect_nonfused_nonprobempi.GLB_T//dfn2_vect_nonfused_nonprobempi.LOC_T)
dfn2_vect_nonfused_nonprobempi["Total_proc"] = dfn2_vect_nonfused_nonprobempi["omp_thrd"]* dfn2_vect_nonfused_nonprobempi.MPI_size
# dfn2_vect_nonfused_nonprobempi.columns

### Data Summary

In [176]:
dfn2_vect_nonfused_nonprobempi_summary = dfn2_vect_nonfused_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn2_vect_nonfused_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,16,16,2,2,2,2,256,114
1,32,8,4,2,2,2,256,69
2,64,4,8,2,2,2,256,35
3,128,2,16,2,2,2,256,17
4,248,1,31,2,2,2,248,4
5,256,1,32,2,2,2,256,7


### Cases Setup

In [177]:
dfn2VectNonFusNonProb_16mpi_16omp = dfn2_vect_nonfused_nonprobempi[(dfn2_vect_nonfused_nonprobempi.omp_thrd == 16) & (dfn2_vect_nonfused_nonprobempi.GLB_T/dfn2_vect_nonfused_nonprobempi.LOC_T == 2) & (dfn2_vect_nonfused_nonprobempi.GLB_X/dfn2_vect_nonfused_nonprobempi.LOC_X == 2) & (dfn2_vect_nonfused_nonprobempi.GLB_Y/dfn2_vect_nonfused_nonprobempi.LOC_Y == 2)& (dfn2_vect_nonfused_nonprobempi.GLB_Z/dfn2_vect_nonfused_nonprobempi.LOC_Z == 2)]
dfn2VectNonFusNonProb_32mpi_8omp = dfn2_vect_nonfused_nonprobempi[(dfn2_vect_nonfused_nonprobempi.omp_thrd == 8) & (dfn2_vect_nonfused_nonprobempi.GLB_T/dfn2_vect_nonfused_nonprobempi.LOC_T == 4) & (dfn2_vect_nonfused_nonprobempi.GLB_X/dfn2_vect_nonfused_nonprobempi.LOC_X == 2) & (dfn2_vect_nonfused_nonprobempi.GLB_Y/dfn2_vect_nonfused_nonprobempi.LOC_Y == 2)& (dfn2_vect_nonfused_nonprobempi.GLB_Z/dfn2_vect_nonfused_nonprobempi.LOC_Z == 2)]
dfn2VectNonFusNonProb_64mpi_4omp = dfn2_vect_nonfused_nonprobempi[(dfn2_vect_nonfused_nonprobempi.omp_thrd == 4) & (dfn2_vect_nonfused_nonprobempi.GLB_T/dfn2_vect_nonfused_nonprobempi.LOC_T == 8) & (dfn2_vect_nonfused_nonprobempi.GLB_X/dfn2_vect_nonfused_nonprobempi.LOC_X == 2) & (dfn2_vect_nonfused_nonprobempi.GLB_Y/dfn2_vect_nonfused_nonprobempi.LOC_Y == 2)& (dfn2_vect_nonfused_nonprobempi.GLB_Z/dfn2_vect_nonfused_nonprobempi.LOC_Z == 2)]
dfn2VectNonFusNonProb_128mpi_2omp = dfn2_vect_nonfused_nonprobempi[(dfn2_vect_nonfused_nonprobempi.omp_thrd == 2) & (dfn2_vect_nonfused_nonprobempi.GLB_T/dfn2_vect_nonfused_nonprobempi.LOC_T == 16) & (dfn2_vect_nonfused_nonprobempi.GLB_X/dfn2_vect_nonfused_nonprobempi.LOC_X == 2) & (dfn2_vect_nonfused_nonprobempi.GLB_Y/dfn2_vect_nonfused_nonprobempi.LOC_Y == 2)& (dfn2_vect_nonfused_nonprobempi.GLB_Z/dfn2_vect_nonfused_nonprobempi.LOC_Z == 2)]
dfn2VectNonFusNonProb_256mpi_1omp = dfn2_vect_nonfused_nonprobempi[(dfn2_vect_nonfused_nonprobempi.omp_thrd == 1) & (dfn2_vect_nonfused_nonprobempi.GLB_T/dfn2_vect_nonfused_nonprobempi.LOC_T == 32) & (dfn2_vect_nonfused_nonprobempi.GLB_X/dfn2_vect_nonfused_nonprobempi.LOC_X == 2) & (dfn2_vect_nonfused_nonprobempi.GLB_Y/dfn2_vect_nonfused_nonprobempi.LOC_Y == 2)& (dfn2_vect_nonfused_nonprobempi.GLB_Z/dfn2_vect_nonfused_nonprobempi.LOC_Z == 2)]


## GFLOPS - NonFused-VECT: Probempi vs NonProbempi

In [178]:
fig = go.Figure() 

fig.add_trace(go.Scatter(x=(dfn2VectNonFusNonProb_256mpi_1omp.Dirc_Data_Mov*(dfn2VectNonFusNonProb_256mpi_1omp.MPI_size/2)), 
                         y=(dfn2VectNonFusNonProb_256mpi_1omp.GFLOPS*(dfn2VectNonFusNonProb_256mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_256mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


fig.add_trace(go.Scatter(x=(dfn2VectNonFusProb_256mpi_1omp.Dirc_Data_Mov*(dfn2VectNonFusProb_256mpi_1omp.MPI_size/2)), 
                         y=(dfn2VectNonFusProb_256mpi_1omp.GFLOPS*(dfn2VectNonFusProb_256mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectFusProb_256mpi_1omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

# case 2
fig.add_trace(go.Scatter(x=(dfn2VectNonFusNonProb_128mpi_2omp.Dirc_Data_Mov*(dfn2VectNonFusNonProb_128mpi_2omp.MPI_size/2)), 
                         y=(dfn2VectNonFusNonProb_128mpi_2omp.GFLOPS*(dfn2VectNonFusNonProb_128mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_128mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn2VectNonFusProb_128mpi_2omp.Dirc_Data_Mov*(dfn2VectNonFusProb_128mpi_2omp.MPI_size/2)), 
                         y=(dfn2VectNonFusProb_128mpi_2omp.GFLOPS*(dfn2VectNonFusProb_128mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectNonFusProb_128mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

# case 3
fig.add_trace(go.Scatter(x=(dfn2VectNonFusNonProb_64mpi_4omp.Dirc_Data_Mov*(dfn2VectNonFusNonProb_64mpi_4omp.MPI_size/2)), 
                         y=(dfn2VectNonFusNonProb_64mpi_4omp.GFLOPS*(dfn2VectNonFusNonProb_64mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_64mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))

fig.add_trace(go.Scatter(x=(dfn2VectNonFusProb_64mpi_4omp.Dirc_Data_Mov*(dfn2VectNonFusProb_64mpi_4omp.MPI_size/2)), 
                         y=(dfn2VectNonFusProb_64mpi_4omp.GFLOPS*(dfn2VectNonFusProb_64mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectNonFusProb_64mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

# case 4
fig.add_trace(go.Scatter(x=(dfn2VectNonFusNonProb_32mpi_8omp.Dirc_Data_Mov*(dfn2VectNonFusNonProb_32mpi_8omp.MPI_size/2)), 
                         y=(dfn2VectNonFusNonProb_32mpi_8omp.GFLOPS*(dfn2VectNonFusNonProb_32mpi_8omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_32mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='red',width=2))))

fig.add_trace(go.Scatter(x=(dfn2VectNonFusProb_32mpi_8omp.Dirc_Data_Mov*(dfn2VectNonFusProb_32mpi_8omp.MPI_size/2)), 
                         y=(dfn2VectNonFusProb_32mpi_8omp.GFLOPS*(dfn2VectNonFusProb_32mpi_8omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectNonFusProb_32mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='green',width=2))))

# case 5

fig.add_trace(go.Scatter(x=(dfn2VectNonFusNonProb_16mpi_16omp.Dirc_Data_Mov*(dfn2VectNonFusNonProb_16mpi_16omp.MPI_size/2)), 
                         y=(dfn2VectNonFusNonProb_16mpi_16omp.GFLOPS*(dfn2VectNonFusNonProb_16mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_16mpi_16omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='chocolate',width=2))))

fig.add_trace(go.Scatter(x=(dfn2VectNonFusProb_16mpi_16omp.Dirc_Data_Mov*(dfn2VectNonFusProb_16mpi_16omp.MPI_size/2)), 
                         y=(dfn2VectNonFusProb_16mpi_16omp.GFLOPS*(dfn2VectNonFusProb_16mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectNonFusProb_16mpi_16omp',
                         line=dict(color='darkviolet',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='crimson',width=2))))



# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"NonFused-VECT (2-Nodes): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1))

## 5) Newgeo_VECT_Probmpi

### Adding Columns

In [180]:
dfn2_vect_newgeo_probempi["N_Nodes"] = 2
dfn2_vect_newgeo_probempi['NPX'] = (dfn2_vect_newgeo_probempi.GLB_X//dfn2_vect_newgeo_probempi.LOC_X)
dfn2_vect_newgeo_probempi['NPY'] = (dfn2_vect_newgeo_probempi.GLB_Y//dfn2_vect_newgeo_probempi.LOC_Y)
dfn2_vect_newgeo_probempi['NPZ'] = (dfn2_vect_newgeo_probempi.GLB_Z//dfn2_vect_newgeo_probempi.LOC_Z)
dfn2_vect_newgeo_probempi['NPT'] = (dfn2_vect_newgeo_probempi.GLB_T//dfn2_vect_newgeo_probempi.LOC_T)
dfn2_vect_newgeo_probempi["Total_proc"] = dfn2_vect_newgeo_probempi["omp_thrd"]* dfn2_vect_newgeo_probempi.MPI_size
# dfn2_vect_newgeo_probempi.columns

### Data Summary

In [181]:
dfn2_vect_newgeo_probempi_summary = dfn2_vect_newgeo_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn2_vect_newgeo_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,16,16,2,2,2,2,256,140
1,32,8,4,2,2,2,256,70
2,64,4,8,2,2,2,256,34
3,128,2,16,2,2,2,256,17
4,248,1,31,2,2,2,248,4
5,256,1,32,2,2,2,256,7


### Cases Setup

In [188]:
dfn2VectNewgeoProb_16mpi_16omp = dfn2_vect_newgeo_probempi[(dfn2_vect_newgeo_probempi.omp_thrd == 16) & (dfn2_vect_newgeo_probempi.GLB_T/dfn2_vect_newgeo_probempi.LOC_T == 2) & (dfn2_vect_newgeo_probempi.GLB_X/dfn2_vect_newgeo_probempi.LOC_X == 2) & (dfn2_vect_newgeo_probempi.GLB_Y/dfn2_vect_newgeo_probempi.LOC_Y == 2)& (dfn2_vect_newgeo_probempi.GLB_Z/dfn2_vect_newgeo_probempi.LOC_Z == 2)]
dfn2VectNewgeoProb_32mpi_8omp = dfn2_vect_newgeo_probempi[(dfn2_vect_newgeo_probempi.omp_thrd == 8) & (dfn2_vect_newgeo_probempi.GLB_T/dfn2_vect_newgeo_probempi.LOC_T == 4) & (dfn2_vect_newgeo_probempi.GLB_X/dfn2_vect_newgeo_probempi.LOC_X == 2) & (dfn2_vect_newgeo_probempi.GLB_Y/dfn2_vect_newgeo_probempi.LOC_Y == 2)& (dfn2_vect_newgeo_probempi.GLB_Z/dfn2_vect_newgeo_probempi.LOC_Z == 2)]
dfn2VectNewgeoProb_64mpi_4omp = dfn2_vect_newgeo_probempi[(dfn2_vect_newgeo_probempi.omp_thrd == 4) & (dfn2_vect_newgeo_probempi.GLB_T/dfn2_vect_newgeo_probempi.LOC_T == 8) & (dfn2_vect_newgeo_probempi.GLB_X/dfn2_vect_newgeo_probempi.LOC_X == 2) & (dfn2_vect_newgeo_probempi.GLB_Y/dfn2_vect_newgeo_probempi.LOC_Y == 2)& (dfn2_vect_newgeo_probempi.GLB_Z/dfn2_vect_newgeo_probempi.LOC_Z == 2)]
dfn2VectNewgeoProb_128mpi_2omp = dfn2_vect_newgeo_probempi[(dfn2_vect_newgeo_probempi.omp_thrd == 2) & (dfn2_vect_newgeo_probempi.GLB_T/dfn2_vect_newgeo_probempi.LOC_T == 16) & (dfn2_vect_newgeo_probempi.GLB_X/dfn2_vect_newgeo_probempi.LOC_X == 2) & (dfn2_vect_newgeo_probempi.GLB_Y/dfn2_vect_newgeo_probempi.LOC_Y == 2)& (dfn2_vect_newgeo_probempi.GLB_Z/dfn2_vect_newgeo_probempi.LOC_Z == 2)]
dfn2VectNewgeoProb_256mpi_1omp = dfn2_vect_newgeo_probempi[(dfn2_vect_newgeo_probempi.omp_thrd == 1) & (dfn2_vect_newgeo_probempi.GLB_T/dfn2_vect_newgeo_probempi.LOC_T == 32) & (dfn2_vect_newgeo_probempi.GLB_X/dfn2_vect_newgeo_probempi.LOC_X == 2) & (dfn2_vect_newgeo_probempi.GLB_Y/dfn2_vect_newgeo_probempi.LOC_Y == 2)& (dfn2_vect_newgeo_probempi.GLB_Z/dfn2_vect_newgeo_probempi.LOC_Z == 2)]


## 6) Newgeo_VECT_NonProbmpi

### Adding Columns

In [189]:
dfn2_vect_newgeo_nonprobempi["N_Nodes"] = 2
dfn2_vect_newgeo_nonprobempi['NPX'] = (dfn2_vect_newgeo_nonprobempi.GLB_X//dfn2_vect_newgeo_nonprobempi.LOC_X)
dfn2_vect_newgeo_nonprobempi['NPY'] = (dfn2_vect_newgeo_nonprobempi.GLB_Y//dfn2_vect_newgeo_nonprobempi.LOC_Y)
dfn2_vect_newgeo_nonprobempi['NPZ'] = (dfn2_vect_newgeo_nonprobempi.GLB_Z//dfn2_vect_newgeo_nonprobempi.LOC_Z)
dfn2_vect_newgeo_nonprobempi['NPT'] = (dfn2_vect_newgeo_nonprobempi.GLB_T//dfn2_vect_newgeo_nonprobempi.LOC_T)
dfn2_vect_newgeo_nonprobempi["Total_proc"] = dfn2_vect_newgeo_nonprobempi["omp_thrd"]* dfn2_vect_newgeo_nonprobempi.MPI_size
# dfn2_vect_newgeo_nonprobempi.columns

### Data Summary

In [190]:
dfn2_vect_newgeo_nonprobempi_summary = dfn2_vect_newgeo_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn2_vect_newgeo_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,16,16,2,2,2,2,256,138
1,32,8,4,2,2,2,256,70
2,64,4,8,2,2,2,256,35
3,128,2,16,2,2,2,256,16
4,248,1,31,2,2,2,248,4
5,256,1,32,2,2,2,256,7


### Cases Setup

In [191]:
dfn2VectNewgeoNonProb_16mpi_16omp = dfn2_vect_newgeo_nonprobempi[(dfn2_vect_newgeo_nonprobempi.omp_thrd == 16) & (dfn2_vect_newgeo_nonprobempi.GLB_T/dfn2_vect_newgeo_nonprobempi.LOC_T == 2) & (dfn2_vect_newgeo_nonprobempi.GLB_X/dfn2_vect_newgeo_nonprobempi.LOC_X == 2) & (dfn2_vect_newgeo_nonprobempi.GLB_Y/dfn2_vect_newgeo_nonprobempi.LOC_Y == 2)& (dfn2_vect_newgeo_nonprobempi.GLB_Z/dfn2_vect_newgeo_nonprobempi.LOC_Z == 2)]
dfn2VectNewgeoNonProb_32mpi_8omp = dfn2_vect_newgeo_nonprobempi[(dfn2_vect_newgeo_nonprobempi.omp_thrd == 8) & (dfn2_vect_newgeo_nonprobempi.GLB_T/dfn2_vect_newgeo_nonprobempi.LOC_T == 4) & (dfn2_vect_newgeo_nonprobempi.GLB_X/dfn2_vect_newgeo_nonprobempi.LOC_X == 2) & (dfn2_vect_newgeo_nonprobempi.GLB_Y/dfn2_vect_newgeo_nonprobempi.LOC_Y == 2)& (dfn2_vect_newgeo_nonprobempi.GLB_Z/dfn2_vect_newgeo_nonprobempi.LOC_Z == 2)]
dfn2VectNewgeoNonProb_64mpi_4omp = dfn2_vect_newgeo_nonprobempi[(dfn2_vect_newgeo_nonprobempi.omp_thrd == 4) & (dfn2_vect_newgeo_nonprobempi.GLB_T/dfn2_vect_newgeo_nonprobempi.LOC_T == 8) & (dfn2_vect_newgeo_nonprobempi.GLB_X/dfn2_vect_newgeo_nonprobempi.LOC_X == 2) & (dfn2_vect_newgeo_nonprobempi.GLB_Y/dfn2_vect_newgeo_nonprobempi.LOC_Y == 2)& (dfn2_vect_newgeo_nonprobempi.GLB_Z/dfn2_vect_newgeo_nonprobempi.LOC_Z == 2)]
dfn2VectNewgeoNonProb_128mpi_2omp = dfn2_vect_newgeo_nonprobempi[(dfn2_vect_newgeo_nonprobempi.omp_thrd == 2) & (dfn2_vect_newgeo_nonprobempi.GLB_T/dfn2_vect_newgeo_nonprobempi.LOC_T == 16) & (dfn2_vect_newgeo_nonprobempi.GLB_X/dfn2_vect_newgeo_nonprobempi.LOC_X == 2) & (dfn2_vect_newgeo_nonprobempi.GLB_Y/dfn2_vect_newgeo_nonprobempi.LOC_Y == 2)& (dfn2_vect_newgeo_nonprobempi.GLB_Z/dfn2_vect_newgeo_nonprobempi.LOC_Z == 2)]
dfn2VectNewgeoNonProb_256mpi_1omp = dfn2_vect_newgeo_nonprobempi[(dfn2_vect_newgeo_nonprobempi.omp_thrd == 1) & (dfn2_vect_newgeo_nonprobempi.GLB_T/dfn2_vect_newgeo_nonprobempi.LOC_T == 32) & (dfn2_vect_newgeo_nonprobempi.GLB_X/dfn2_vect_newgeo_nonprobempi.LOC_X == 2) & (dfn2_vect_newgeo_nonprobempi.GLB_Y/dfn2_vect_newgeo_nonprobempi.LOC_Y == 2)& (dfn2_vect_newgeo_nonprobempi.GLB_Z/dfn2_vect_newgeo_nonprobempi.LOC_Z == 2)]


## GFLOPS - Newgeo VECT: Probempi vs NonProbempi

In [192]:
fig = go.Figure() 

fig.add_trace(go.Scatter(x=(dfn2VectNewgeoNonProb_256mpi_1omp.Dirc_Data_Mov*(dfn2VectNewgeoNonProb_256mpi_1omp.MPI_size/2)), 
                         y=(dfn2VectNewgeoNonProb_256mpi_1omp.GFLOPS*(dfn2VectNewgeoNonProb_256mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_256mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


fig.add_trace(go.Scatter(x=(dfn2VectNewgeoProb_256mpi_1omp.Dirc_Data_Mov*(dfn2VectNewgeoProb_256mpi_1omp.MPI_size/2)), 
                         y=(dfn2VectNewgeoProb_256mpi_1omp.GFLOPS*(dfn2VectNewgeoProb_256mpi_1omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_256mpi_1omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

# case 2
fig.add_trace(go.Scatter(x=(dfn2VectNewgeoNonProb_128mpi_2omp.Dirc_Data_Mov*(dfn2VectNewgeoNonProb_128mpi_2omp.MPI_size/2)), 
                         y=(dfn2VectNewgeoNonProb_128mpi_2omp.GFLOPS*(dfn2VectNewgeoNonProb_128mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_128mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn2VectNewgeoProb_128mpi_2omp.Dirc_Data_Mov*(dfn2VectNewgeoProb_128mpi_2omp.MPI_size/2)), 
                         y=(dfn2VectNewgeoProb_128mpi_2omp.GFLOPS*(dfn2VectNewgeoProb_128mpi_2omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_128mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

# case 3
fig.add_trace(go.Scatter(x=(dfn2VectNewgeoNonProb_64mpi_4omp.Dirc_Data_Mov*(dfn2VectNewgeoNonProb_64mpi_4omp.MPI_size/2)), 
                         y=(dfn2VectNewgeoNonProb_64mpi_4omp.GFLOPS*(dfn2VectNewgeoNonProb_64mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_64mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))

fig.add_trace(go.Scatter(x=(dfn2VectNewgeoProb_64mpi_4omp.Dirc_Data_Mov*(dfn2VectNewgeoProb_64mpi_4omp.MPI_size/2)), 
                         y=(dfn2VectNewgeoProb_64mpi_4omp.GFLOPS*(dfn2VectNewgeoProb_64mpi_4omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_64mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

# case 4
fig.add_trace(go.Scatter(x=(dfn2VectNewgeoNonProb_32mpi_8omp.Dirc_Data_Mov*(dfn2VectNewgeoNonProb_32mpi_8omp.MPI_size/2)), 
                         y=(dfn2VectNewgeoNonProb_32mpi_8omp.GFLOPS*(dfn2VectNewgeoNonProb_32mpi_8omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_32mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='red',width=2))))

fig.add_trace(go.Scatter(x=(dfn2VectNewgeoProb_32mpi_8omp.Dirc_Data_Mov*(dfn2VectNewgeoProb_32mpi_8omp.MPI_size/2)), 
                         y=(dfn2VectNewgeoProb_32mpi_8omp.GFLOPS*(dfn2VectNewgeoProb_32mpi_8omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_32mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='green',width=2))))

# case 5

fig.add_trace(go.Scatter(x=(dfn2VectNewgeoNonProb_16mpi_16omp.Dirc_Data_Mov*(dfn2VectNewgeoNonProb_16mpi_16omp.MPI_size/2)), 
                         y=(dfn2VectNewgeoNonProb_16mpi_16omp.GFLOPS*(dfn2VectNewgeoNonProb_16mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_16mpi_16omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='chocolate',width=2))))

fig.add_trace(go.Scatter(x=(dfn2VectNewgeoProb_16mpi_16omp.Dirc_Data_Mov*(dfn2VectNewgeoProb_16mpi_16omp.MPI_size/2)), 
                         y=(dfn2VectNewgeoProb_16mpi_16omp.GFLOPS*(dfn2VectNewgeoProb_16mpi_16omp.MPI_size/2)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_16mpi_16omp',
                         line=dict(color='darkviolet',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='crimson',width=2))))



# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"Newgeo VECT (2-Nodes): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1))